<a href="https://colab.research.google.com/github/increpare/tatoeba_toki_pona_spellcheck/blob/main/tatoeba_toki_pona_spellcheck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install emoji

# by jan Inkepa - https://github.com/increpare/tatoeba_toki_pona_spellcheck/blob/main/tatoeba_toki_pona_spellcheck.ipynb

import os
import urllib.request
from IPython.display import Markdown, display
from emoji import UNICODE_EMOJI

print("downloading toki pona sentences from tatoeba")
urllib.request.urlretrieve('https://downloads.tatoeba.org/exports/per_language/toki/toki_sentences_detailed.tsv.bz2', 'toki_sentences_detailed.tsv.bz2')

!rm -rf /content/toki_sentences_detailed.tsv

print("decompressing data")
!bunzip2 /content/toki_sentences_detailed.tsv.bz2
print("done")


     |████████████████████████████████| 51kB 2.8MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49716 sha256=bb91df687dca40e4f628adc9c2f6ded8510f73eb446b5fb1064d31ea4ba86c14
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji
downloading toki pona sentences from tatoeba
decompressing data
done


In [2]:
import pandas as pd
df = pd.read_csv("/content/toki_sentences_detailed.tsv",names=["k","v","user","added","modified"],delimiter="\t")
words = ["a","akesi","ala","ali","alasa","ale","anpa","ante","anu","awen","e","en","esun","ijo","ike","ilo","insa","jaki","jan","jelo","jo","kala","kalama","kama","kasi","ken","kepeken","kili","kin","kipisi","kiwen","ko","kon","kule","kulupu","kute","la","lape","laso","lawa","leko","len","lete","li","lili","linja","lipu","loje","lon","luka","lukin","lupa","ma","mama","mani","meli","mi","mije","moku","moli","monsi","monsuta","mu","mun","musi","mute","namako","nanpa","nasa","nasin","nena","ni","nimi","noka","o","oko","olin","ona","open","pakala","pali","palisa","pan","pana","pi","pilin","pimeja","pini","pipi","poka","poki","pona","pu","sama","seli","selo","seme","sewi","sijelo","sike","sin","sina","sinpin","sitelen","sona","soweli","suli","suno","supa","suwi","tan","taso","tawa","telo","tenpo","toki","tomo","tonsi","tu","unpa","uta","utala","walo","wan","waso","wawa","weka","wile"]

In [6]:
import re

def has_emoji(s):
    count = 0
    for emoji in UNICODE_EMOJI:
        count += s.count(emoji)
        if count >= 1:
            return True
    return False

def validate(w):
  if w[0].isupper():
    return True#w.lower() not in words
  return w in words or w.isnumeric()

errors={}

#checks for "mi/sina" li sentences, and roughly for sentences with an "e" but without a "li"/"o"/"mi"/"sina"
def validate_sentence(s,tokens,index,user):
  if user not in errors:
    errors[user]=[]

  if (tokens[0]=="mi" or tokens[0]=="sina") and tokens[1]=="li":
    errors[user].append([s,index,"mi\sina li"])
    pass
  if "li" not in tokens and "e" in tokens and "o" not in tokens:
    if "mi" not in tokens and "sina" not in tokens:      
      errors[user].append([s,index,"'e' without 'li'/'mi'/'sina'/'o'"])
      pass
  if bool(re.search(r"\b(la,?|\.) (mi|sina) li\b",s)):
    errors[user].append([s,index,"mi\sina li"])
    pass
  if bool(re.search(r"(wu|wo|ji|ti)",s)):
    errors[user].append([s,index,"forbidden syllables wu/wo/ji/ti"])
    pass
  lastchar = s[len(s)-1]
  if lastchar.isalnum():
    errors[user].append([s,index,"Sentence doesn't end with punctuation"])
    pass
  if has_emoji(s):
   errors[user].append([s,index,"Contains emoji"])

freq={}
links={}
id_table={}

from ipywidgets import IntProgress
from IPython.display import display
f = IntProgress(min=0, max=len(df),description="searching...") # instantiate the bar

display(f) # display the bar


i=0
for index, row in df.iterrows():
  import re
  id_table[index]=row
  user = row['user']
  sentence=row['v']
  tokens = re.findall(r'\w+',sentence)
  validate_sentence(sentence,tokens,index,user)
  for w in tokens:
    if not validate(w):
      if ('"'+w+'"') not in sentence:
        if w not in freq:
          freq[w]=0
          links[w]=[]
        freq[w]=freq[w]+1
        links[w].append(index)
        errors[user].append([sentence,index,"likely misspelled word: "+w])
  i=i+1
  if (i%1000)==0:
          f.value=i

import datetime

output="# Tatoeba Toki Pona Error Report\n\n"+str(datetime.datetime.now())+"\n\n"

def printm(s):
  global output
  output=output+"\n"+s;

for u in errors:
  errorlist = errors[u]
  if len(errorlist)==0:
    continue
  printm("\n")
  printm("***")
  printm(" \n")
  printm("### Likely Error report for user "+u)
  printm("\n")
  for error in errorlist:
    printm(""+error[0]+"  ")
    printm("- "+str(error[2])+"  ")
    printm("- http://tatoeba.org/eng/sentences/show/"+str(error[1])+"  ")
    printm("")

display(Markdown(output))

IntProgress(value=0, description='searching...', max=41716)

# Tatoeba Toki Pona Error Report

2020-11-25 14:58:11.953896




***
 

### Likely Error report for user kroko


„ona li open!“ – „ala. sina li open!“  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/2969963  

sina wile ala wile wan e nanpa ni  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/3005286  

mi pona e tomo pi pata mi.  
- likely misspelled word: pata  
- http://tatoeba.org/eng/sentences/show/3005615  

pona! lawa insa sina li pini tawa sina pi nanpa tu. sina li kama moli li pona.  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/3080251  

pata mi li jo e soweli.  
- likely misspelled word: pata  
- http://tatoeba.org/eng/sentences/show/3938433  

mi kama jo e kili Apo teka tu lon esun.  
- likely misspelled word: teka  
- http://tatoeba.org/eng/sentences/show/4600833  



***
 

### Likely Error report for user bojnin


jan lili mute li musi lon ma kasi pi ma tomo  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/642739  

tenpo suno kama la seli li pona la mi tawa tan insa  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/667636  

pilin pi kasi kule Lila li lon ma kasi pi poka tomo mute  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/740208  



***
 

### Likely Error report for user negativeclock


jan mute li olin e jan ni pi kalama pona sin ala  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/1419652  



***
 

### Likely Error report for user Tepan


ni li suli ala tawa mi: sina pilin e lon ona anu seme  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/7798319  

ona li pilin ike tan kama mi  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/7833816  

tenpo suno ni la ona kama sona e jan pona sin lon tomo sona.  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/8041562  

Mi pana e nimi "Kesi" tawa ona.  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/8318475  

jan Ton li kama sona e ni: ken al jan Mewi li toki e ijo suli tawa ona.  
- likely misspelled word: al  
- http://tatoeba.org/eng/sentences/show/9061013  

jan ton li tawa musi ala.  
- likely misspelled word: ton  
- http://tatoeba.org/eng/sentences/show/9093391  

sina wila, la sina ken awen lon poka mi lon ma tomo Pasen.  
- likely misspelled word: wila  
- http://tatoeba.org/eng/sentences/show/9099265  

jan ton li jo e ilo tawa pona.  
- likely misspelled word: ton  
- http://tatoeba.org/eng/sentences/show/9109313  

jan ton li lawa e tomo tawa suli.  
- likely misspelled word: ton  
- http://tatoeba.org/eng/sentences/show/9146248  

jan Wope Teniwo li kama lon sitelen tawa ni lon tenpo lili.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/9159880  

Wir werden das zusammen tun.  
- likely misspelled word: werden  
- http://tatoeba.org/eng/sentences/show/9188160  

Wir werden das zusammen tun.  
- likely misspelled word: das  
- http://tatoeba.org/eng/sentences/show/9188160  

Wir werden das zusammen tun.  
- likely misspelled word: zusammen  
- http://tatoeba.org/eng/sentences/show/9188160  

Wir werden das zusammen tun.  
- likely misspelled word: tun  
- http://tatoeba.org/eng/sentences/show/9188160  

jan ton li jo e ilo pi pona uta lon tomo pali ona. kepeken ona la ona li pona e uta ona lon pini moku.  
- likely misspelled word: ton  
- http://tatoeba.org/eng/sentences/show/9195993  

jan wile pali pi musi ala e ni.  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/9265889  



***
 

### Likely Error report for user shekitten


ni li nimi pi jan Kowele. mama ona li jan Tawi, li lan lawa pi ma tomo Jelusalen.  
- likely misspelled word: lan  
- http://tatoeba.org/eng/sentences/show/3445158  



***
 

### Likely Error report for user Wadimiy


pata suli li lukin e sina.  
- likely misspelled word: pata  
- http://tatoeba.org/eng/sentences/show/2907834  



***
 

### Likely Error report for user rkqb


tempo suno kama la jan Kise li ken ala pali.  
- likely misspelled word: tempo  
- http://tatoeba.org/eng/sentences/show/1703362  



***
 

### Likely Error report for user carlosalberto


jan lawa nanpa wan pi kule pimeja pi ma Mewika li jan Palakawopama.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/4543481  

jan Awowi li tawa musi pona mute.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/4545010  

tawa lon ma kasi li pona tawa sina au seme?  
- likely misspelled word: au  
- http://tatoeba.org/eng/sentences/show/4558156  

tenpo seme la sina li kama sona e tawa lon telo?  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/4602325  

tenpo suno nanpa wan pi tenpo sike kama, la sina li lon seme?  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/4640207  

mi sona e ni: sina kama sona mute  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/4640786  

jan Sunewoki Iketa li pona tawa ona.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/4674880  

jan Siwokene li jan sona pi ma Elena li jo ala e tomo. tomo ona li poki sike pi tenpo suli.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/4712141  

tenpo suno pini la sina li toki tawa jan Ton?  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/4722722  

ilo sitelen ni anu ani li suli mute?  
- likely misspelled word: ani  
- http://tatoeba.org/eng/sentences/show/4737812  

mi wile e ni: tenpo ale la sina li pilin pona.  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/4737856  

jan Ton li wile ala tawa kepken noka lon tenpo ni.  
- likely misspelled word: kepken  
- http://tatoeba.org/eng/sentences/show/4893861  

tenpo pini la sina li jan pona mi.  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/4916520  

sinpin pi tenpo pimeja pini la sina li lon seme?  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/4939437  

tan ni la sina li pona tawa mi.  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/4940814  

nasin pona ii pona mute ala.  
- likely misspelled word: ii  
- http://tatoeba.org/eng/sentences/show/9135950  

jan mute li kona e kipisi ni pi ma tomo Palisi.  
- likely misspelled word: kona  
- http://tatoeba.org/eng/sentences/show/9146915  

mi pani e ilo open lon supa.  
- likely misspelled word: pani  
- http://tatoeba.org/eng/sentences/show/9199213  

jan Ton li kama sona e toki Kanse lon tenpo mun du wan.  
- likely misspelled word: du  
- http://tatoeba.org/eng/sentences/show/9310171  



***
 

### Likely Error report for user soweli_Elepanto


sina tawa lili taso la sina li kama moli.  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/1722700  

jan Losi li musi uta ala  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/1725107  

moku en len sielo en tomo li suli mute tawa jan ali.  
- likely misspelled word: sielo  
- http://tatoeba.org/eng/sentences/show/7270281  

jan Ton taso li jan pi pona sielo lon ma tomo.  
- likely misspelled word: sielo  
- http://tatoeba.org/eng/sentences/show/8013499  

jan Ton li lon ni la sina li toki e seme tawa ona?  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/8076636  

mije mama pi jan Ton li jan pi pona sielo.  
- likely misspelled word: sielo  
- http://tatoeba.org/eng/sentences/show/8302047  

plilin mi la mi nasa lukin.  
- likely misspelled word: plilin  
- http://tatoeba.org/eng/sentences/show/8622476  



***
 

### Likely Error report for user ToinhoAlam


mi wile ala kute e pali sina pi tempo suno pini.  
- likely misspelled word: tempo  
- http://tatoeba.org/eng/sentences/show/4900154  

tempo pimeja la mun li kama.  
- likely misspelled word: tempo  
- http://tatoeba.org/eng/sentences/show/4976921  

tempo pini la jan Mewi li meli pi jan Ton.  
- likely misspelled word: tempo  
- http://tatoeba.org/eng/sentences/show/5139181  



***
 

### Likely Error report for user komap


mo jo e palisa luka mute.  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/9140426  

mo jo e palisa luka mute.  
- likely misspelled word: mo  
- http://tatoeba.org/eng/sentences/show/9140426  



***
 

### Likely Error report for user temlat


telo Waloti li telo nasa ala. ona li telo kasi seli.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/2622442  



***
 

### Likely Error report for user klagefall


mi en meli mi li wan  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/3002378  

kute tan kama sona. kute tan ala toki  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/3002384  



***
 

### Likely Error report for user Reonaato


jan pona kute e jan sewi.  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/6634557  



***
 

### Likely Error report for user pinaxe


mi pilin e ni: tenpo kama li pona ala anu li pona lili  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6174026  

ijo in li ijo sina ala sina?  
- likely misspelled word: in  
- http://tatoeba.org/eng/sentences/show/6177177  



***
 

### Likely Error report for user list


soweli pi mu "uuu-aaa-a-a-a" li jo e sona wawa.  
- likely misspelled word: uuu  
- http://tatoeba.org/eng/sentences/show/6058474  

soweli pi mu "uuu-aaa-a-a-a" li jo e sona wawa.  
- likely misspelled word: aaa  
- http://tatoeba.org/eng/sentences/show/6058474  

mi wile olin e soweli 🐕 sina la mi ken ala ken kepeken luka?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6332503  

jan 💂 li lon ma.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6333424  

jan Mawi li moku e kili loje Tomato  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6372734  

soweli 🐕 ni li sona kin!  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6374442  

mi wile sona e ni: tenpo pini la tan seme la jan Ton li kama jo lon esun e ni  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6376397  

jan Ton li jo e sijelo ike. ona li toki sama waso pimeja Kowo.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/6376658  

tenpo ale ona la ona pana e pona tawa jan.  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/6376694  

tenpo muno li jo e tenpo suno li mute seme?  
- likely misspelled word: muno  
- http://tatoeba.org/eng/sentences/show/6381347  

akesi 👻 li ken tawa lon kon!  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6390260  

akesi sewi 👻 li ken jo e jan.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6390306  

mi kepeken musi ♟ Sako la mi pana e sona lon sona insa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6390419  

mije li toki lon la mije ni li wile tawa wawa kepeken soweli 🏇.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6391224  

jan ni li weka ike e sitlen lape pi mi mute la jan ni li lawa e moli tawa mi mute.  
- likely misspelled word: sitlen  
- http://tatoeba.org/eng/sentences/show/6391233  

jan 🤠 li wile kama sona. taso sina sama soweli 🐃 la sina li kama sona pi wawa mute.  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/6391330  

jan 🤠 li wile kama sona. taso sina sama soweli 🐃 la sina li kama sona pi wawa mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6391330  

jan wawa 🏋️‍♂️ kin pi mute ale li lukin kin lon anpa pi supa lape 🛏.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6391357  

pip Luso seme li tawa lon insa Epato sina?  
- likely misspelled word: pip  
- http://tatoeba.org/eng/sentences/show/6392362  

sina pana e mani 💶 la ni li pona tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6397122  

tenpo suno kama ni la ko lete li kama anpa la mi pali e sijelo ☃ pi ko lete.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6397222  

jan li toki tan akesi ike suli 👹 la jan Kati li kama.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/6397359  

jan li toki tan akesi ike suli 👹 la jan Kati li kama.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6397359  

mi jan suli la mi wile kama jan 👨‍⚕️ pi sijelo pona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6397376  

tenpo kama ni la kon sewi li pona la mi mute li kepeken sike musi ⚾ tawa musi Pasepa.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6397423  

mi pilin e ni la mi pilin 🥶.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6398856  

suwi 🎂 li kama ala pona la seme li lon?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6398910  

sina kepeken ilo sike wawa 🏍 la o kepeken selo lawa ⛑.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6398980  

soweli 🐈 li moli la soweli 🐀 li moku.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6399041  

jan li toki tan soweli 🐯 la soweli 🐅 li kama.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6399060  

mi jan suli la mi kama jan 👩‍🚀‍🚀.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6399068  

soweli 🐫 li jo e luka la ona li pali.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6399098  

mi jan suli la mi kama jan lawa 👑.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6400478  

sina olin kin e mi la o olin 😗 e mi kepeken uta 💋.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6401583  

sina kute e ilo 🚨 pi kalama wawa la o tawa kepeken noka. o tawa ala kepeken noka wawa.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6401621  

sina pakala ala e jan ante la sina ken pali e wile sina  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6401642  

ona li moku ala la ona li kama pilin 😥.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6404626  

ona li kepeken telo nasa mute la tenpo ale la ona li pilin 😢.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6404665  

ona mije li kepeken telo nasa mute la tenpo ale la ona meli li pilin 😢.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6404677  

ma tomo Situtika li kama pakala la seme li lon?  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/6406004  

jan Mawi li jan pali pi sitlen tawa li jo e ken.  
- likely misspelled word: sitlen  
- http://tatoeba.org/eng/sentences/show/6452080  

jan Mawi li pilin 😨 tan soweli 🐶.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6457090  

pona la moku e moku pona e telo pona la ike la toki nasa.  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/6457104  

tomo lipu 🏭📰 li kama moli la nasin Temokati li kama moli.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/6466795  

tomo lipu 🏭📰 li kama moli la nasin Temokati li kama moli.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6466795  

jan Tane pana e wan lili tan suwi pimeja tawa jan Linta.  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/6467180  

tenpo ala la jan Tane li kama sona ala e mama sina  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6467190  

tenpo wan pini la  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6469732  

kiwen 🗿 li sewi lon telo la lipu 🍂 li kama anpa lon telo.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6471442  

mi mije li olin 👬 e mije la seme li lon?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6471505  

sina wile toki e ijo la o lawa e lukin mi kepeken luka 👋 sina.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6471670  

kili pomo li lon ni. mi lukin e ona.  
- likely misspelled word: pomo  
- http://tatoeba.org/eng/sentences/show/6480018  

olin li wile ala kama jo tan jan ante. olin li wille pana e ali tawa jan ante.  
- likely misspelled word: wille  
- http://tatoeba.org/eng/sentences/show/6480036  

jan li toki e ni: sina ike la ijo li pona tan sina. jan li utala taso e jan ni: ona li jo e sike musi ⚽.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6480136  

o weka e ale lon sona sina. telo li kiwen ala li sijelo ala. - o kama sama telo. jan li pana e telo 💧 lon poki 🍵 la telo 💧 li sama poki 🍵. telo li ken tawa li ken pakala. jan pona mi o sama telo.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6480137  

mi mute li pali e anu tawa ni: kama weka lon lape la ona li ken pali e sitlen lape ona tawa lon.  
- likely misspelled word: sitlen  
- http://tatoeba.org/eng/sentences/show/6482929  

mi open e poki toki 📻 la ni li ike ala ike tawa sina?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6485417  

jan li seli 💥 pi lili mute tan suno pi mama sewi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6532842  

tenpo sinpin la sina kepeken e ilo sitlen tawa la o pali e pali pi kama sona.  
- likely misspelled word: sitlen  
- http://tatoeba.org/eng/sentences/show/6534496  

sinpin la suwi lete 🍦 li kama telo la o moku e ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6538536  

sinpin la sina kepeken ilo utala 🔫 la o lukin pona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6538543  

sinpin la sina tawa supa lape la o moku ala e telo nasa 🍺.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6540194  

sitelen pi nimi ma li selo tan palisa pi kasi suli la jan li ken lili lukin e sitlen ni.  
- likely misspelled word: sitlen  
- http://tatoeba.org/eng/sentences/show/6544005  

mi pilin sama ni: lawa 🤯 mi li kama pakala 💥.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6551083  

mi jan Anitile.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/6578583  

o kepeken ala luka tawa poki 👜 mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6586017  

o kepeken ala luka tawa lipu 📖.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6586023  

jan lili 👶 li kepeken luka tawa ale la ni li pona tawa ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6586091  

sina o kepeken ala luka tawa lipu 📕.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6586093  

tan seme la sina kepeken luka tawa meli ❤ mi?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6586108  

luka sina li jaki la o kepeken ala luka tawa lipu 📗 mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6586185  

o kepeken ala luka tawa poki 🍳 ni. ona li seli mute!  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6586211  

jan 👮👮‍♀️ li kama jo e jan 🧟‍♀️ pi nasin lawa ala lon luka.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6586228  

sina wile kepeken luka tawa lete 🧊 la o kepeken ala luka taso.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6586257  

akesi ike suli 👹 li kama jo e wan pi linja lawa sina la tenpo ale la sina li jo pi akesi ike suli.  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/6587590  

akesi ike suli 👹 li kama jo e wan pi linja lawa sina la tenpo ale la sina li jo pi akesi ike suli.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6587590  

mi wile pilin e soweli 🐮 kepeken luka la mi kepeken len luka 🧤 tan ni: ona 🐄 li ken ala pakala e mi kepeken uta.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6587592  

o pilin e tawa pi pilin 💓 mi lon luka mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6589289  

lon pi mel ni li suwi.  
- likely misspelled word: mel  
- http://tatoeba.org/eng/sentences/show/6590967  

jan Putin li wile e nasin seme pi kulupu jan li lawa tawa lon?  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/6595743  

mi wile ala pakala e jan ali  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6596965  

tan seme la mi lawa e poki mani 👛 sina tawa jo mi?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6601257  

mute seme pi len noka anpa la sina li jo?  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/6602498  

tenpo monsi pi tenpo sike pi tu wan la meli ni li tawa weka tan jan olin la tan ni la jan oli li kepeken ike e meli ni.  
- likely misspelled word: oli  
- http://tatoeba.org/eng/sentences/show/6617669  

sina kepeken pilin 😢 la ike ni li kama ike lili.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6617814  

insa pi jan ale li lon sike kule sewi 🌈. jan li wile lukin e ona 🌈 la jan li ken lukin e ona 🌈!  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6620149  

ni li waso 🐔.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6626820  

nimi mi li jan Lutiwi.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/6626836  

o pini e suno 🕯 kepeken kon.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6630033  

mi kepeken kon tawa sike kon 🎈 mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6630040  

jan ni li olin li lawa e ijo 🎁 suli.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6633968  

o pana e len lon palisa kepeken linja. o tawa sewi e ona 🏁. sina kama lukin e ni: jan mute li tawa lon monsi pi sitelen len ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6634079  

seme li kama? sina pilin 😢 tan seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6638611  

sina wile e ni la o kama jo e kili 🍌🍒.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6640049  

open ala. tan ni la pona e tomo.  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/6667614  

wile la walo moku Salito tawa e seli moku.  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/6669160  

lon seme la kama e pakala?  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/6682102  

mi kama jo ala e sono sin tan jan Tone.  
- likely misspelled word: sono  
- http://tatoeba.org/eng/sentences/show/6688317  

pon sina o lon seme li poki telo jaki?  
- likely misspelled word: pon  
- http://tatoeba.org/eng/sentences/show/6700528  

lon seme la ken lukin e sitelen tawa?  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/6700617  

lon seme la sina li tomo tawa?  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/6700663  

mi sona e tomo esun pi wawa ilo ⛽ lon ni lon poka lili.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6702635  

mi lukin lon sona mi la sina li mije sama mi.  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/6702656  

mi jo e pan 🥪 e kili 🍎 lon poka mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6751109  

tan seme la jan lili li pilin 😭 mute kin?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6830526  

ona 👩 li pona uta lili 🙂 la pilin 💘 mi 😍 li seli 🔥.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6830550  

tenpo lili la tenpo Alowin 🎃 li kama.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6843409  

sijelo pi jan Ton li suli mute la sielo pi jan Mawi li suli lili.  
- likely misspelled word: sielo  
- http://tatoeba.org/eng/sentences/show/6843418  

sike kule sewi 🌈 li kule.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6847878  

jan Ton li pana e pono tawa mama meli ona lon tomo moku.  
- likely misspelled word: pono  
- http://tatoeba.org/eng/sentences/show/6847920  

suno li sono walo mute kin la mi mute li wile e lukin tawa poka.  
- likely misspelled word: sono  
- http://tatoeba.org/eng/sentences/show/6851919  

tenop suno ni la mi wile ala musi e musi Teniso.  
- likely misspelled word: tenop  
- http://tatoeba.org/eng/sentences/show/6855920  

ni li pona lukin lon sitelten. ni li pona lili lon lon.  
- likely misspelled word: sitelten  
- http://tatoeba.org/eng/sentences/show/6862159  

telo 💧 pi lili mute li kama pi sin en sin la lupa li kama lon kiwen.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6866951  

sina awen ala sona e ni: o pini e selo noka 👞 sina kepeken linja.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6882027  

mi wile ala e ni: ona mijel li kama tawa ma pi akesi Satan. taso mi ken ala weka e ike tan ona.  
- likely misspelled word: mijel  
- http://tatoeba.org/eng/sentences/show/6886624  

jan Ton li kama sin la ona li kama jo e pan 🥖 lon esun.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6886701  

pona li ni: tomo sin en ja pona pi tenpo suli.  
- likely misspelled word: ja  
- http://tatoeba.org/eng/sentences/show/6891252  

sina kepeken e nena lili ni la lupo tomo li kama open.  
- likely misspelled word: lupo  
- http://tatoeba.org/eng/sentences/show/6892286  

ona li tomo lon tomo pi mani mute. poka lili lon ma laso kulupu Senta Pawo.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/6893979  

telo 🥛 li lon supa anpa la pilin 😢 li lawa ala.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6894082  

jan li kekepen kasi tawa supa sitelen.  
- likely misspelled word: kekepen  
- http://tatoeba.org/eng/sentences/show/6895331  

mi jan suli la mi wile kama jan 🧑‍🚒🚒.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6897278  

tenpo pini la mi lape pona ala  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6897354  

sin Pipon 🧂 li pona tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6899848  

ilo tawa mi li tenpo suli mute la mi li tenpo suli lili.  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/6899857  

tenpo mute la jan suli mute Atila li lukin sama e ni: ona li jo e sike ma lon sijelo Sulito.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/6899877  

ilo ✂ li lon seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6900144  

tawa mi la tenpo mun pi nanpa luka tu tu li mute ale lon tenpo sike  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6900215  

mije lawa mi li moli e akesi 🐉. ona li wile e ni: mi toki e ni: o pana e pan 🍞 sama ni: jan lawa 👑 li moku.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6903800  

mije lawa mi li moli e akesi 🐉. ona li wile e ni: mi toki e ni: o pana e moku 🥩 sama ni: jan lawa 👑 li moku.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6903803  

mije lawa mi li moli e akesi 🐉. ona li wile e ni: mi toki e ni: o pana e kili 🥦🥕 sama ni: jan lawa 👑 li moku.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6903804  

mije lawa mi li moli e akesi 🐉. ona li wile e ni: mi toki e ni: o pana e suwi 🧁🍭 sama ni: jan lawa 👑 li moku.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6903805  

mije lawa mi li moli e akesi 🐉. ona li wile e ni: mi toki e ni: o pana e telo nasa 🍷 sama ni: jan lawa 👑 li moku.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6903806  

soweli 🐶 li mu la soweli 🐶 ni li pakala ala kepeken uta.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6911072  

sina kepeken len 🤡 la ona mute li ken ala lukin e ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6918061  

sike ma 🌍 ni li sona mute. ona li jo e telo mute sama ilo telo 🚿 pi poki jaki 🚽.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6920170  

ni li pona: telo nasa 🍺 la monsi la telo nasa 🍷. ni li ike: telo nasa 🍷 la monsi la telo nasa 🍺.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6921710  

ni li ike: telo nasa 🍷 la monsi la telo nasa 🍺. ni li pona: telo nasa 🍺 la monsi la telo nasa 🍷.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6921712  

jan li toki e ni: nimi a. la ona li wile toki kin e ni: nimi p. ona li pana lon tenpo wan la ona li wile pana kin lon tenpo tu.  
- likely misspelled word: p  
- http://tatoeba.org/eng/sentences/show/6931594  

tenpo ala la toki e ni: ala!  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/6931709  

tomo tawa telo li kama anpa la soweli lili 🐀 li tawa weka.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6933749  

sina ken ala kepeken e toki lon ala tawa mi. mi lukin e ni: tenpo pini la jan pali utala li lon ni kin. taso ona li kama jo ike ala e ijo. sina li jan pi kama jo ike. sina wile pana e mani.  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/6934826  

mi sona e ni: sina jo ala e ijo ni. mi kama pana e mani mute tawa sina. taso mi wile sona e ni: ken la sina li jan pi pilin akesi la mi wile ala pana e mani tawa ala.  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/6934830  

ona 👨 li lon lupa 🥅.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6941618  

jan 👨⚓ li jo e sike lukin 👀 laso la ni li pona kin tawa mi mute 👭.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6941656  

toki olin taso la kepeken e sike pi palisa luka.  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/6943316  

jan seme li kama pilin 😢 tan jan wan ni?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6946121  

sina jo ala e ijo 🎞 kule pi ilo 📷 poka sina.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6946209  

lon seme la tomo wawa ⛽ pi tomo tawa lon poka lili?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6947934  

telo sijeo loje pi tu wan li tawa anpa lon supa pi ko lete.  
- likely misspelled word: sijeo  
- http://tatoeba.org/eng/sentences/show/6955304  

waso li kama tawa lukin e jan lili. waso Siko e waso Kowo e waso Kolonpo.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/6955423  

pona nasa la lupa telo li jo ala e telo. jan pi tenpo sike lili li tawa anpa pakala lon kasi lili Muko taso. taso ona li ken tawa ala tan lupo.  
- likely misspelled word: lupo  
- http://tatoeba.org/eng/sentences/show/6963589  

suno lili taso tawa lukin e lipu.  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/6981367  

telo loje siejelo li lon len sina anu seme?  
- likely misspelled word: siejelo  
- http://tatoeba.org/eng/sentences/show/6981492  

o kiwen e ni lon kasi suli 🌳 kepeken linja.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6994276  

jan seme la ken pini e ni?  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/6994307  

tenpo wawa la soweli lili 🐀 li kama mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6994310  

tenpo wawa la soweli lili 🐇 li kama mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6994314  

walo sewi 💭💭 li lon sewi la telo sewi 🌧 li kama.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6996296  

wawa lili sina seme la sina li jo?  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/6996679  

telo sewi 🌧 la mute seme pi telo 💧 li kama anpa?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6998030  

sike kule sewi 🌈 li suli seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/6998040  

tan seme la sina li jan pi pona ale tawa pali ni?  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/7001171  

tenpo mute la sina li ike sijelo anu seme?  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/7001334  

sina pilin 😭 tan seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/7001489  

jan li wile weka ike e ilo 🚲 mi la mi ken pali e seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/7007090  

tawa jan ante la mi jan lawa. taso mi ala kin lon kupupu.  
- likely misspelled word: kupupu  
- http://tatoeba.org/eng/sentences/show/7009772  

ale ll jaki lon sike mi.  
- likely misspelled word: ll  
- http://tatoeba.org/eng/sentences/show/7020066  

sina lukin ala e pona lili pi tenpo lili lo o pini.  
- likely misspelled word: lo  
- http://tatoeba.org/eng/sentences/show/7030039  

tawa jan Ton la jan Mali li ike lukin. taso jan Mali li ken kama lon pona lukln.  
- likely misspelled word: lukln  
- http://tatoeba.org/eng/sentences/show/7039112  

nasin seme la sike kule sewi 🌈 li kama lon sewi?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/7042394  

jan Toni en jan Ton li utala sama e jan Tawi en jan Koli  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/7054245  

temo ala la jan Mali li moku e kiwen insa en selo pi kili Pomo.  
- likely misspelled word: temo  
- http://tatoeba.org/eng/sentences/show/7059362  

sina jo ala jo e lete pi ike siejelo?  
- likely misspelled word: siejelo  
- http://tatoeba.org/eng/sentences/show/7109069  

jan Ton li open e kalama musi uta ona  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/7115418  

pilin sina la sina li meli pi pona ale lukin lon sike ma.  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/7128361  

tenpo suo open ale la jan Ton li awen lon ma kasi.  
- likely misspelled word: suo  
- http://tatoeba.org/eng/sentences/show/7223669  

tan seme la sina pilin 😢 kin?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/7247159  

meli li suli  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/7247231  

tenpo suno ni la jan Mali li seli e moku sike Patikuko.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7255562  

o moku e pan Pano sins poka ko jelo Putelo lon pini.  
- likely misspelled word: sins  
- http://tatoeba.org/eng/sentences/show/7259801  

telo naso Wino loje ni li pona ale tawa mi.  
- likely misspelled word: naso  
- http://tatoeba.org/eng/sentences/show/7372885  

sina wile ala wile moku e kili pomo? - wile ala. mi moku e kili tu lon tenpo pini.  
- likely misspelled word: pomo  
- http://tatoeba.org/eng/sentences/show/7461691  

tenpo 🎄 la mi li tawa tomo pi jan lili mi.  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/7477348  

tenpo 🎄 la mi li tawa tomo pi jan lili mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/7477348  

tenpo 🎄 la jan ale pi kulupu mama li kama wan.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/7477352  

tawa moku la jan Ton li moku e telo naso pi poki tu taso.  
- likely misspelled word: naso  
- http://tatoeba.org/eng/sentences/show/7477703  

soweli li anpa lo supa monsi la sina ken ala ken lukin e ona?  
- likely misspelled word: lo  
- http://tatoeba.org/eng/sentences/show/7487904  

mi awen ala sona e ni: mi wile kama jo e pan 🍞 lon esun.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/7526602  

ona 👨 li pona ala e ilo 🔪🔪.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/7555074  

jan li pali e ni: poki mani 👛 mi li kama weka lon tomo tawa kulupu 🚍.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/7645417  

tomo tawa ni li jaki la oni li wile e pona.  
- likely misspelled word: oni  
- http://tatoeba.org/eng/sentences/show/7666648  

wile mir la ken la sina sama li ken toki e ale tawa mi.  
- likely misspelled word: mir  
- http://tatoeba.org/eng/sentences/show/7680675  

sina wile e mani 💶 la o toki tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/7684693  

o tawa e poki 📦 tawa ma pi poki ante 📦.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/7703258  

jan pi poka seme la sina li lon?  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/7703411  

ona li kama sona e toki Latino lon tomo sona.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7713814  

pilin mi la sina kepeken e len ni la sina li pona lukin.  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/7758284  

nasina ala tawa ni: jan li ken weka tan lon ni.  
- likely misspelled word: nasina  
- http://tatoeba.org/eng/sentences/show/7768376  

sina moku ala molu e telo nasa Wino? - ala. mi kepeken ilo tawa la mi moku e telo kili Pomo.  
- likely misspelled word: molu  
- http://tatoeba.org/eng/sentences/show/7786322  

jan li lukin e ni: luka 🤚 pi jan Ton li jo e ijo ni: jan Ton li kepeken ilo utala 🔫.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/7799794  

tenpo pini la sina li jan pali pi tomo moku anu seme?  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/7816890  

ilo supa ⛓ li lon ala lon tawa jan pi kama pona?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/7817145  

mi wile e nasin ni: tan tomo awen pi jan kama tawa tomo awen pi tomo tawa lijna.  
- likely misspelled word: lijna  
- http://tatoeba.org/eng/sentences/show/7817169  

jan Ton li wile ala kepeken ilo pi sitele tawa.  
- likely misspelled word: sitele  
- http://tatoeba.org/eng/sentences/show/7818990  

nasin seme la nasin Politiko li pali lon ma Kanse?  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7820227  

sina lukin sama e ni: sina pilin nasa la tan seme la sina li ni?  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/7821481  

mi will pona e jan Ton tan ni: ona li ken tawa tomo sona.  
- likely misspelled word: will  
- http://tatoeba.org/eng/sentences/show/7829288  

tenpo seme la sina kama lon suno? - tenpo mun pi nanpa tu tu la tenpo suno pi luka wan  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/7848143  

o tenpo pimjea open pona tawa meli en mije mi. o kama pona lon sitelen tawa pi mi mute.  
- likely misspelled word: pimjea  
- http://tatoeba.org/eng/sentences/show/7848213  

mi lukin e ni: sina pilin 😢 la ni li ike mute tawa pilin 💔 mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/7866277  

tenpo ilo pi napa tu wan la jan Ton li kama lon tomo awen pi tomo tawa linja.  
- likely misspelled word: napa  
- http://tatoeba.org/eng/sentences/show/7874828  

nimi ni li kama tan toki Latin.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7874896  

seme li ma Palatiso tawa sina?  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7883926  

m tawa tomo mi. tenpo lili la mi kama sin.  
- likely misspelled word: m  
- http://tatoeba.org/eng/sentences/show/7887619  

poki ☕ wan li lon supa moku.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/7887635  

mi lon poka pi nasin Temokati.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7887868  

nasin Temokati li pona tawa mi.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7887869  

tan seme la sina kepeken e supa mons mi?  
- likely misspelled word: mons  
- http://tatoeba.org/eng/sentences/show/7894628  

tomo 🏨🧳 pi mi mute li poka lili lon ma laso Senta Pak.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/7916493  

suwi pan 🥮 li lon seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/7919228  

mi kute pona e ni la jan Ton li wan 👩‍❤️‍👨 poka meli.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/7919239  

ilo ni pi sitele tawa li pakala.  
- likely misspelled word: sitele  
- http://tatoeba.org/eng/sentences/show/7919275  

sama ni la sina wile kama jo e mi la mi li kiwen tawa sina.  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/7928258  

jan 👨‍🌾 li jo e soweli 🐕 pona. nimi ona 🐕 li nimi Sutan. ona li tenpo suli. ona li jo ala e walo uta 🦷.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/7931654  

sina tawa esun la o kama sin poka pan 🍞.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/7953653  

mi pilin sin sama ni: mi jan pi tenpo sike lili  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/7960997  

sina ala lon ni la tan ni: mi mute li pilin ike  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/7965476  

mi lukin e ona 👩‍🦰 lon tomo ☕. tomo ☕ ni li poka lili lon tomo 🚃.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/7978824  

telo mute li lon sike ma 🌎.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/7992790  

jan Ton li wile la ona li ken kepeken e ilo 📹 mi pi sitelen tawa.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8168820  

sitelten sona ona li suli ala.  
- likely misspelled word: sitelten  
- http://tatoeba.org/eng/sentences/show/8178509  

wawa pi kon tawa suli Tapuno 🌪 li kama kin lon lili.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8178816  

o tawa. sina sama o luki pali.  
- likely misspelled word: luki  
- http://tatoeba.org/eng/sentences/show/8178970  

o tenpo pimjea pona. o sitelen lape tawa sina.  
- likely misspelled word: pimjea  
- http://tatoeba.org/eng/sentences/show/8182663  

tenpo suno pini ni la mije sama pi mama mi li moli tan ike sielo Kanselo.  
- likely misspelled word: sielo  
- http://tatoeba.org/eng/sentences/show/8184588  

unu. tu, tu wan. tu tu. luka. luka wan. luka tu. luka tu wan. luka tu tu. luka luka.  
- likely misspelled word: unu  
- http://tatoeba.org/eng/sentences/show/8198974  

akesi 🐊 li lon nasin telo ni anu seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8209701  

sina lon sina la sina li pona tawa mi.  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/8209740  

tenpo ale la mi moku e telo nasa 🍺.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8218831  

ja Lisa li wile ala lon kulupu.  
- likely misspelled word: ja  
- http://tatoeba.org/eng/sentences/show/8220556  

tenpo pi ike sijo ona la jan pona li kama tawa ona lon tenpo mute.  
- likely misspelled word: sijo  
- http://tatoeba.org/eng/sentences/show/8220603  

sina sama e ni la sina li pona tawa mi.  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/8223348  

telo siejlo loje li telo pi pona mute.  
- likely misspelled word: siejlo  
- http://tatoeba.org/eng/sentences/show/8225254  

a! ona li kalam musi ike! jaki! ona li kama musi pi nasin lawa!  
- likely misspelled word: kalam  
- http://tatoeba.org/eng/sentences/show/8225289  

kon nasa Tapako 🚬 en telo nasa Lun 🥃 la sina pilin pona la moli li kama.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8230951  

jan 😌 li kama jo e ijo la pilin pi jan ante 😡😡 li ni: jan 😔 li weka e ijo ni tan jan ante 😡😡.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8230996  

teno suno sin ale la jan Ton li pali lukin e ijo ante.  
- likely misspelled word: teno  
- http://tatoeba.org/eng/sentences/show/8306566  

mi awen ala sona e ni: mi wile kepeken ilo toki 📞 tawa ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8326412  

lipu 💌 ni li lon tawa sina.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8329001  

kili 🍌 li lon ala lon ni?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8333186  

kili 🍌 li lon ala lon?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8333189  

o pana e ilo 🔪 tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8333221  

tenpo ala la jan Ton li toki eni.  
- likely misspelled word: eni  
- http://tatoeba.org/eng/sentences/show/8333292  

tenpo Alowin 🎃 li ike mute tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8338959  

ilo toki ☎ li lon tomo mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8423490  

sina pilin 😨 la o open e suno.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8486495  

tenpo mute la mije sama mi li awen lape ala lon tenpo pimjea.  
- likely misspelled word: pimjea  
- http://tatoeba.org/eng/sentences/show/8612710  

ona li jo ala e wawa pi sitlene sona.  
- likely misspelled word: sitlene  
- http://tatoeba.org/eng/sentences/show/8612757  

jan tawa pi ma ante li kute e toki pi jan lawa  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/8621360  

sina lukin ala lukin e sitele lawa pi ona mije?  
- likely misspelled word: sitele  
- http://tatoeba.org/eng/sentences/show/8622798  

ona li seli e suwi pan Mupin  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/8622807  

pan 🍕 li tawa lon nasin.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8622963  

jan Ton li tawa e poki 🍵 pi telo kasi seli tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8664279  

tenpo suno napa pi luka wan li tenpo suno musi ona.  
- likely misspelled word: napa  
- http://tatoeba.org/eng/sentences/show/8666221  

teno ni la jan seme li lon sewi?  
- likely misspelled word: teno  
- http://tatoeba.org/eng/sentences/show/8666320  

sina sona ala sona wawa e ni: tomo tawa lijna ni li pona?  
- likely misspelled word: lijna  
- http://tatoeba.org/eng/sentences/show/8670991  

tomo moku ni li pona mute. taso ike li ni: ma 🅿 ala li lon poka lili.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8673982  

kili mute lon pokl.  
- likely misspelled word: pokl  
- http://tatoeba.org/eng/sentences/show/8677007  

mi lon seme lon nasin ni. o lawa e lukin mi  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/8688368  

mama mama meli mi o lawa e lukin mi tan ni: nasin seme la mi li pona e supa lape mi?  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/8690719  

mi awen ala sona e ni: ni li lon lipu nanpa 🗒.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8749594  

wawa pi ilo sona 🖥 ni li kama tan poki 🔋.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8886427  

jan Ton li moku sama soweli 🐷.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8886468  

kili 🍎 anu kili 🍌 li pona tawa sina?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8886501  

o kama jo e suwi lete 🍦lon esun.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8886504  

tomo 🏠 ni li tomo sina ala. taso tomo mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8886507  

mama mije mi li kule e poki lipu 📮 kepeken loje.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8886514  

ona li seli e suwi pan 🍪.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8886522  

linja loje pi uta sina li sama selo pi kasi 🌵.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8886529  

kili 🍏 ni li suwi ala lon uta anu seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8886545  

wan pi nena kute 👂🏻 ona li kute ala.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8886553  

mi tawa tomo sewi ⛪.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8886561  

jan Mali li kama loje sama kule 🔥.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8886563  

soweli 🦓 ni li jo sina anu seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8886572  

poki ni li lon tawa ilo moku 🍴.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8886575  

mi pona e ilo sitelen ✏️ mi kepeken ilo 🔪.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8886577  

mi kama tawa sina poka kasi kule 🌹 loje.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8886579  

seme li insa lon poki monsi 🎒 ona?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8886581  

mi jo e ko 🧈 taso lon poki lete mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8886582  

jan Ton li pana e sin kasi Olekano lon pan 🍕.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8886586  

jan Ton li tawa ma tomo 🏙️ kepeken tomo tawa kulupu 🚌.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8886596  

mi telo e kasi kule 🥀.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8886603  

kili li kiwen la kasi suli 🌳 li kama anpa.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8886620  

sina lon ni la o kepeken ilo toki 📱.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8886635  

tenpo ni la sina tawa la sina kama pona tawa tomo tawa linja 🚂.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8886636  

ilo tawa sike 🚲 mi li loje.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8886637  

tomo tawa telo 🚢 li tawa weka tan ma awen.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8886638  

jan li wile sona e tomo awen 🏨 pona la mi ken toki e ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8886640  

jan Mali li pona e linja pi soweli 🐎 ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8886643  

ilo toki 📱 laso li lon supa sitelen.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8886645  

tomo tawa linja 🚂 li pona mute tawa ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8886657  

ona li ken pona tawa lon telo 🏊. taso ona li ken pona musi e musi Teni 🏸 kin.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8888617  

mi jan ala ni: ona li sitelen e lipu ✉️ ni lon tenpo pini.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8888634  

ona li pana e ilo open 🗝️ tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8888656  

ona li wile e tomo tawa 🚗 sin.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8888660  

mi pana e ilo musi 🎸 lon esun la ni li kiwen tawa mi. taso mi wile e mani.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8888672  

mi kama jo e lipu 🧻ni taso: ona li ko mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8888687  

tenpo pini la jan 👮‍♂️ li awen e jan Ton lon luka ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8888697  

mama meli mi li lukin pali e telo walo 🥛 kepeken uta.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8888705  

tomo 🏠 ni li jo ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8888723  

o kepeken ala sin 🧂 mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8888781  

telo sewi 🌧️ li pini kin.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8888833  

soweli 🐈 la o sina pali e seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8888890  

sina la pona o pana sin e telo pimeja ☕ tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8888907  

len lawa 🎩 ni li lili pi mute ike tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8888916  

jan Ton li pilin e ni: jan 👮 li kepeken toki lon ala.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8888921  

tan seme la sina wile ala kama jo e tomo tawa 🚗 lon esun?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8888922  

jan Ton li open e poki 📦. ona li lukin lon insa.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8888925  

ni li poki 👜 mi ala.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8888927  

nena sinpin 👃 pi jan Ton li loje.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8889157  

soweli mute li moku e moku soweli 🍖 sama soweli 🦁.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8889203  

ona li open e ni: ona li tawa sike e sike ma 🌎 kepeken ilo tawa telo ⛵.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8889215  

sina pali e seme kepeken ilo 📷 mi?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8889229  

tenpo wawa mute la soweli 🐕 ni li tawa kepeken noka.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8889271  

mi wile wawa e ilo toki 📱 mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8889294  

mi kama jo e lipu 📰 tawa ona lon esun.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8889313  

suwi lete 🍦 ni li pona mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8889323  

lipu 📰 ni li pona anu seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8889335  

len 👖 ni li pona lukin lon sijelo sina.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8889375  

sina kepeken len 👖 ni la ni li pona lukin.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8889377  

mi meli 👩‍⚕️ li pali lon tomo 🏥.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8889398  

ona li pilin 😨 tan soweli 🐩.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8889428  

jan li toki suwi tawa akesi 👹 la akesi 👹 li pona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8890790  

tenpo pini la jan lawa 🤴 li lon tomo 🏰 pi tenpo suli.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891196  

o weka ala e poki 👜 sina.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891251  

mi tawa e tomo tawa 🚗 tan monsi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891309  

telo sewi 🌧️ li pona tawa ona 👩.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891318  

lipu ken 🎫 pi tomo tawa linja 🚂 li mani mute anu seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891343  

ona 👩 li lawa e tomo tawa 🏎 pi jelo wawa.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891352  

ilo suno 🚨 li loje! akesi 🤖 li kama!  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891364  

tomo tawa 🚘 li kama.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891366  

mi pilin 😨 tan jan 👨‍⚕️ 🦷.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891382  

walo 🦷 li kama weka tan uta pi jan Ton.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891385  

jan 👮‍♂️👮‍♀️ li kama lon ni. mi mute o tawa weka.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891387  

mi lawa e ilo tawa sike 🚲 la tepo ale la mi kepeken selo lawa ⛑.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891402  

mi lawa e ilo tawa sike 🚲 la tepo ale la mi kepeken selo lawa ⛑.  
- likely misspelled word: tepo  
- http://tatoeba.org/eng/sentences/show/8891402  

mu pi waso 🐦 🦜 li pona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891410  

jan pali 👨‍🔧 li pona e tomo tawa 🚙 pi mi mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891413  

pona o weka e supa monsi 🪑 tawa poka. ona 🪑 li lon nasin mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891415  

nimi pi kasi suli 🌳🌲🌴 lon ni li seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891457  

tomo 🛁 🚽 li sewi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891475  

sinpin pi tenpo seli la ona 🧔 li pana e sike lili pi kasi kule 🌻 lon ma.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891500  

ona 👱 li pana e pan 🍞 e telo 🥛 pi soweli 🐂 tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891522  

selo noka 👟 pi jan Ton li anpa lon supa lape 🛏 ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891531  

mi sitelen e lipu olin 💌 tawa ona 👩.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891538  

ona li anu e ni: ona li wile kama jan lawa 👨‍✈️ pi tomo tawa kon ✈.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891569  

jan Ton li kama jo e kasi kule 💐 tawa jan Mawi lon esun.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891595  

mi sona ala e jan lon tomo sona 🏫 ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891607  

o kama tan supa lape 🛏.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891639  

ona 👩‍🦰 li lon ilo telo 🚿.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891654  

ken la lipu 📕 ni li jo e sona ni: sina wile sona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891660  

ona li pana e len 👔 laso tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891710  

mije kala 🧜‍♂️ 🧜‍♂️ li lon anpa pi telo suli.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891731  

ilo sitelen 🖋 mi li pakala.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891739  

jan Mawi li lukin e waso 🐤 🐦 la ni li pona tawa ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891761  

suno 🌄 li kama weka la mun 🍪 li kama.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891767  

sitelen tawa 🎞 ni li pona tawa sina kin.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891772  

ona li moku e kili 🍓 e ko walo la ni li pona pi mute ale tawa ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891789  

sina ken ala ken kepeken ilo toki 📞 tawa ona?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891797  

akesi 🐸 pi noka wawa li moku e pipi 🐛🕷🐜.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891801  

suno 🌞 li suno tawa jan pona en jan ike.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891803  

kiwen luka 💅 li suli la ni li pona tawa meli ale.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8891811  

sina kama kepeken tomo tawa linja 🚅 seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8893816  

tenpo pona la tomo tawa linja 🚅 pi mi mute li kama lon ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8893818  

nasin seme la jan li kama jo e pan 🍞 sin pi pona mute ale?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8893820  

jan Ton li sona e ni: ilo utala 🔫 li lon luka pi jan Mali la ilo utala ni li ilo musi taso.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8893835  

akesi 🐉 li lon lupa pi nena suli ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8893845  

linja lawa pi mama mije 👨‍🦳 li kama walo.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8893850  

suwi pan 🧀 ni li suwi pi mute ike.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8893858  

sina wile e telo pi kili 🍊 anu seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8893862  

tenpo seme la tomo tawa linja 🚄 ni li open e tawa ona?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8893864  

ilo tenpo 🕐 mi en ilo tenpo 🕑 sina li toki e tenpo pi sama ala.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8893868  

o seli 🍳 e sike waso 🥚 tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8893879  

jan li ken ala lukin e mi lon sitelen 🖼 tan ni: mi kepeken ilo 📷.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8893883  

sina kama jo e waso 🦜 lon esun.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8893890  

ma awen 🚏 pi tomo tawa 🚍 li lon poka pi tomo 🏨.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8893905  

tenpo suno insa pini la mi moku e pan 🌭.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8893932  

jan Ton li kepeken ilo musi 🎿 la noka ona li kama pakala.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8893936  

ijo ala li lon poki 🛄 pi jan Ton.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8893943  

moku kala 🍣 li pona ala pona tawa sina?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8893944  

ona 💂 li moli e akesi 🐲 la ona 💂 li selo e meli lili 👸 pi jan lawa 👑.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8893958  

sina lukin pali kin e suwi 🍯 ni lon uta anu seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8893963  

mi moku e kili 🍓.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8893971  

lipu 📧 li lon ni li tawa sina.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8893985  

mi seli 🍳 e moku 🥞 poka kili 🍄.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8894013  

mi mute li tawa poka 🏖 pi telo suli. mi mute li wile tawa lon telo 🏊.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8896013  

jan li lawa e tomo tawa 🚙 la ona li wile sona e lawa ni: ona li wile pali e seme lon nasin pi tomo tawa.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8896083  

tenpo kama la mi kule 🎨 e tomo 🏠 mi pi kule pimeja walo kepeken kule laso 💦.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8896108  

kon tawa 🌬 li mama e telo tawa 🌊.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8896117  

jan utala 💂 ni ale li kepeken len sama.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8896144  

o telo e noka 👣 sina tan ni: jaki li weka.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8896149  

ken la jan li weka e poki mani 💰 pi meli sama lili 👧 mi lon tomo sona 🏫.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8896165  

meli sama suli 👩 mi li pali lon kulupu ‍👩‍🚒 ni: ona li telo 🧯e seli 🔥.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8896172  

pipi 🕷 en pipi 🦂 li kulupu mama sama.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8896182  

jan lili 🧒👱 li toki e ni tawa mi: suwi lete kili 🍨🍓 li pona tawa ona 👱🧒.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8896186  

soweli 🐈 li mama e soweli lili 😸😽😾 pi tu wan.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8896201  

mi moku e suwi 🍭🍫🍬.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8897690  

kasi suli 🌳 li pona la ona 🌳 li ken ala jo e kili 🍑 ike.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8897700  

jan Ton li jo e pali. ona li jan pali 👨‍🔧.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8897733  

jan Ton li jan 🧑‍🚀🚀.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8897757  

o lawa e ni: jan ‍👨‍🚒🚒 li kama.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8897761  

jan 🦸‍♂️ li ken tawa lon kon.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8897765  

jan Ton li kepeken ilo musi 🎸. jan Mali li kalama uta musi 👩‍🎤.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8897807  

ona li lukin e akesi 🐍 suli ni la ona li pilin 😱.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8897818  

pilin pi jan Ton la ona li wile e len lawa 🎩.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8897824  

sina mute o kepeken selo noka 👞.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8897830  

tenpo pini la akesi 👹 pi ike mute li tomo lon ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8897846  

tomo tawa 🚑 li tawa kepeken ilo kalama 📢 en 🚨 laso.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8897873  

soweli 🐐 li moku e kasi lili 🌱 lon ma.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8897884  

telo 🥛 pi soweli 🐄 li pona ala pona tawa pona sijelo?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8897927  

poki toki 📻 li pakala la sina ken pona ala pona e ona?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8897937  

jan Ton li jo e tomo tawa kon ✈ pi ona sama.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8898000  

mi selo e poki 🍸 lon lipu 🗞 tan ni: ona li ken ala pakala.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8898041  

o telo e walo uta 🦷 ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8898069  

jan Ton li pana e pona ni tawa mi: ona li pona e ilo tenpo ⏲ mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8898258  

mi mute li wile e pan 🍝 e pan 🍕 tu.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8898284  

sina kama ala kama jo e kasi kule 💐 lon tenpo pini?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8898288  

jan Ton li lukin e palisa pona pi kasi suli 🌳 lon poka ⛱ pi telo suli.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8898320  

jan Ton li tawa e mi tawa ma awen 🚏 pi tomo tawa kulupu 🚍.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8898338  

o pana e lipu 📗 sina tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8898356  

ona 👩‍🦰 li pana e mani 💷 tawa tomo mani 🏛.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8898366  

mi toki e ni tawa jan Ton: ona li ken kepeken tomo len ⛺ lon ma kasi 🏕 pi mi mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8898380  

o tawa weka tan supa lape 🛌 mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8898410  

tenpo pimeja open ni la jan li ken lukin e mun sike 🌝.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8898422  

sina ken ala 🚫 kepeken ilo nasa 🚬 lon ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8898428  

jan Ton li moku e kili 🍓.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8898439  

mi mute li moli e soweli 🐀 kepeken palisa.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8898479  

tenpo ilo 🕕 pi luka wan la mi mute li tawa kepeken tomo tawa kon.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8898485  

jan seme li sitelen e lipu 📙 ni?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8898490  

jan Ton li pana e pona tawa mama meli ona. ona li lawa e ilo tawa 🛒.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8898510  

sina kepeken ilo nasa 💊 sina anu seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8898518  

ona li lawa e ni: jan 🕵️‍♂️ li kama.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8898625  

mi wile kama jo e kili 🍇 anu kili 🍓 lon esun?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8898629  

jan Ton li toki e ni: ona li jan sewi 🥇.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8898632  

o pana e len 🧥 tawa ilo len.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8898636  

jan Ton li pana e telo nasa 🍾 tawa poki 🍷 pi jan Mali.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8898638  

tenpo pini lili la mi kama jo e moku ni lon esun. mi pana e moku soweli 🥩 tawa poki lete.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8898661  

kala 🐬 li kala 🐠 ala sama soweli 🐕.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8898686  

soweli lili 🐿 mute li lon ma ni pi kasi suli kulupu 🌲🌲🌲.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8898696  

meli lili 👯‍♀️💃 pi tu wan li tawa musi la jan Emi 💃 li jan pi nanpa wan.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8898700  

ona li toki e ni tawa jan lon esun: o kama jo e len linja 👔 laso.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8900115  

tomo esun 🏬 ona li kama pakala. ona li wile e mani 💶 la ona li wile pana e tomo ona lon esun la ona li kama jo e mani.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8900178  

suwi 🍭 🍩 li kama la mi ken ala pini e mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8900182  

ona li jan 🧙‍♀️.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8900183  

jan Mawi li kepeken tomo tawa 🚘 la ona li kama pakala 🤕.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8900215  

tenpo seli ni la mi mute 👫 li wile kama wan 👰👦.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8900242  

kon tawa 🌬 li lon ni: ona li wile.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8900269  

jan Ton li lon ilo tawa telo 🚤 la ona li kama ike sijelo 🤮.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8900282  

nena sinpin 👃 ona li lukin sama e kili 🥔.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8900289  

tomo tawa 🚙 li tawa la ona li kepeken telo wawa ⛽.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8900304  

mi kama jo e sike 💍 ni pi nena kute 👂 tan mama mama meli 👵 mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8900306  

tenpo pimeja 🌌 la insa moku li kama pi tu tu tan uta mi 🤮.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8900312  

mi tawa tomo sewi ⛪ la mi weka e len lawa 🎩 mi tan lawa 🙆.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8900379  

mi pona uta mute 🤣 kin la poka lili la suwi 🍬 li lawa e mi tawa moli.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8900436  

kon sewi 🌫 lete ni li pona mute tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8900476  

tenpo 🎃 la jan Ton li kepeken len 👙 pi meli lili.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8900510  

soweli lili 🐀 mute li lon ma jaki pi ma tomo.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8900526  

mi kepeken ilo musi 🎸.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8900530  

kasi kule 🌹 li lon supa moku pi jan seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8900583  

mani 💵 mute li sama jaki 💩 pi tomo soweli 🐃. jan li pana e ni tawa ale la ona li pona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8900609  

jan 👨‍🌾 li sona ala e kili la ona li moku ala e kili.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8900789  

mi tawa anpa. mi tawa e soweli 🐶 tawa sewi lon luka 💪 mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8900867  

tenpo suli seme la tomo esun li open lon tenp suno ni?  
- likely misspelled word: tenp  
- http://tatoeba.org/eng/sentences/show/8900890  

ona li moli. pimeja ⚱ ona li lon ma ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8900914  

sina kin pilin 😱 sama mi tan ni anu seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8900949  

mi jo ala e mani pi mute pona tawa ni: mi kama jo e tomo tawa 🚗 sin.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8900966  

sina kama ala kama jo e pan 🍞 lon esun?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8900968  

walo sewi 💭 pi wan ala li lon sewi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8900992  

tenpo suno Suno la mama meli li seli e pan 🍪.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8901070  

mi mute li wile e ilo ni: ona li moli e pipi 🦟.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902279  

jan Ton li pana e telo 🍾 lon poki 🍷 ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902287  

musi mi li kepeken ilo 📷.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902299  

sina ken musi pi pona mute e musi 🎾 Teni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902301  

jan Ton li ala lon sitelen 🖼 ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902308  

poki wawa 🔋 mi li kama pini.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902321  

tenpo suno ni ale la jan Ton li lape lon poki 🏖 pi telo suli.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902335  

ken la ona li ken ala kepeken tomo tawa linja 🚅 ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902338  

mi mute li tawa lon nasin la tomo tawa 🚘 pi mi mute li kama pakala.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902344  

ken la soweli 🦮 li wile moku.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902349  

mi pilin e lukin ona 👩‍🦰 lon monsi mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902373  

ko lete 🌨 suli li kama.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902377  

mi kepeken len 🧦 suli la ni li pona tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902380  

jan Ton li pilin 😢 tan ni: mama mama meli 👵 ona li moli.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902387  

ni li kasi suli 🌳 kili 🍊 ala.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902480  

ona li toki e ni: meli lili li olin e ona kepeken uta 😚.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902516  

tenpo ale la ona li tawa 🏃 lon ma laso kepeken noka.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902591  

jan li ken lukin taso e selo 👤 pi ona meli lon walo kon 🌫.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902593  

jan Ton li kepeken ilo pona 🧼 en telo 🚰 tawa luka 🤚 ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902598  

sike ma 🌎 li suno ⭐ ala. taso ona li sike ma 🪐.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902600  

jan Ton li pana e pan 🌾 tawa soweli 🐴 ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902601  

o weka ala e ni lon sona sina: o kepeken len 👔.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902621  

lon li anpa e mi mute la mi mute li tawa musi 💃🕺 kepeken musi Linpo.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902626  

tenpo pimeja ale la ona li moku e telo nasa 🍺🥃 mute la ona li ken lape taso.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902643  

mi tawa 🏊‍♀️ lon nasin telo la ni li musi tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902653  

sike lukin 👁 pi waso 🐧 li lukin pona lon anpa telo.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902665  

mi wawa mute 🏋.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902667  

ona en mama ona 👪 li tawa tomo pi sitelen tawa 🎞.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902676  

lipu 📰 li toki ala e sona sin ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902679  

ilo musi 🎸 ni li mani mute ni: mi ken ala kama jo e ona 🎸 lon esun.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902681  

mi kama jo e ilo 📷 sin.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902682  

mi moku e moku 🥗🥔 la ni li pona tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8902687  

ona 🤢 li pilin ike tan moku 🥘.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8904953  

soweli 🐱 li moku e telo 🥛 pi soweli 🐄.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8904989  

mi moku e moku 🍟 la ni li moku pi nanpa wan mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905000  

ilo ⛱ ni li lukin sama lili e ilo ☂ mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905014  

tomo sona 🏫 tu li lon ma tomo lili pi mi mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905038  

pilin mi la jan Ton 👨‍🦱 li lukin sama jan Son 👨‍🦰.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905046  

jan Ton li kepeken supa monsi 🪑 lon poka pi ilo musi 🎹 la ona li open e kalama musi 🎵.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905063  

ona li jo e mani pi mute pona tawa ni: ona li ken kama jo e sitelen 🖼 lon esun.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905075  

sina lon tomo 🎭 la sina lukin e jan seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905081  

ona li selo lon anpa pi supa lape 🛏.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905086  

tomo 🏠 ni li mani 💵 seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905094  

sitelen sona ni li kama tawa mi la mi pilin 😨!  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905105  

tenpo pimeja open pini ni la seli 🔥 li open lon tomo esun 🏪.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905123  

soweli 🐿 li tawa wawa tan palisa tawa palisa pi kasi suli 🌲.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905129  

ilo nasa 💉 ni li lawa e ni: jan 🤕 li kama pilin 😑.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905137  

ona 👧 li kute e ni: ona 🧔 li pilin 😭 lon tenpo pimeja insa.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905158  

ona li lukin pali e ni: lupa tomo 🚪 li kama open. taso ona li lukin e ni: ni li ken ala.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905159  

jan mute li kama moli tan ilo nasa 💊. ona li kama moli ala tan ike sijelo ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905171  

akesi 🧚‍♀️ li lon lipu 📕 taso.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905175  

o weka ala e ilo open 🗝 ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905180  

akesi 🐸,🐸 en akesi 🦎 li akesi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905197  

ona 😶 li ken ala toki e nimi wan.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905205  

selo li weka la mije lili 🧟‍♂️ jaki li kama mije lili 🤴 pi jan lawa 👑.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905215  

o pilin 😱 ala.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905220  

pipi 🦋 mute li tawa sike lon kasi 🌿.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905321  

tenpo monsi la telo sewi 🌧 la kon li kama lete.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905383  

mama mije ona li jan 👮‍♂️.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905428  

sina kepeken ilo musi 🎹 la ni li pona ala pona?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905439  

jan Ton en jan Mali 👫 li tawa lon ma laso kepeken noka.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905448  

selo noka 🥾 sina li ike lukin tawa len 👘👗 ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905465  

meli 👩‍🦳 li kule e selo tomo kepeken walo.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905519  

jan pona ona li jan pi ma 🇹🇷.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905523  

sina ken lili e kalama pi poki toki 📻 anu seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905545  

mi weka e ilo open 🔑 mi lon ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905547  

ilo sitelen 🖍 sina li kule seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905560  

meli lili 👧 li pana e ilo open 🔑 lon poki 👜 ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905608  

ona li jo e tomo tawa 🚐 pi ona sama.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905616  

pipi 🕷 suli li lon monsi pi supa ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905651  

linja lawa pi jan Ton 👨 li kama weka tan lawa 👨‍🦲 ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905662  

soweli 🐿 li alasa mute e kili 🥥🥜.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905698  

ona li toki e ni: ona li moku ala e soweli 🐷.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905702  

insa 💓 mi li wawa ala.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905706  

tenpo suno mute la seli 🔥 li pakala e ma pi kasi suli 🌲🌲🌲.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905712  

waso 🐔 li lon sike 🥚 lon tomo 🏚 lon kasi seli.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905721  

kili 🍌🍌 ni li kama jaki.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905726  

tenpo mute la jan pona mi li pona e linja lawa mi 💇‍♀️.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905756  

tenpo lili la lete 🧊🧊 ni li kama telo.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905784  

tenpo suli ala la jan 🧑‍🚀 li ken tawa mun 🌔.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905788  

jan 👩‍⚕️ li weka e walo 🦷 pakala lon uta mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905795  

ona 👨‍🦱 li pona e monsi ona 👩‍🦱 kepeken luka.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905798  

tenpo suli seme la sina wile awen lon ma 🏝 ni?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905817  

jan Ton en jan Mali li jan tu 🧑👧 li wile e selo noka 👡👟 sin.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905828  

ko lete ❄ li selo e ma kasi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905837  

mi lukin ala e ilo open 🔑 pi tomo tawa 🚙.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905840  

jan Mawi li awen e insa lawa 🧠 pi jan Ton lon poki.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905846  

mama mama meli 👵 mi li seli e moku 🍲 tawa jan 👭👪 luka.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8905851  

mije ni li utata e ona. ona li wile moli e ona.  
- likely misspelled word: utata  
- http://tatoeba.org/eng/sentences/show/8908424  

o weka e soweli 🐈 tan supa 🛋.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8908444  

soweli 🐱 ni li jo pi jan Ton.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8908463  

o pana e ilo open 🔑 tawa mi tan ilo 🔒 ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8908491  

jan Ton li jan 😨.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8908498  

jan Ton li kepeken ilo 🕛 pi jelo mani lon luka.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8908500  

mi moku e kili 🍐 ala.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8908502  

wile mi la mi ken kama jo e ilo musi 🎸 ni lon esun.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8908503  

mije lili 👦 li utala e akesi 🐸 kepeken kiwen.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8908523  

tan seme la sina wile e ilo 📺 sin?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8908536  

jan Ton li lon poka pi kasi suli 🌳 kili 🍎.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8908553  

jan Mali li kama jo e supo monsi ni: ona li poka lili lon lupa tomo.  
- likely misspelled word: supo  
- http://tatoeba.org/eng/sentences/show/8908570  

tomo tawa linja 🚂 li tawa ala tan ko lete ❄.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8909136  

mi wile e ni: ona 👩 li kama jan sewi 🥇.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8910711  

mi lawa e ilo open 🔑 lon lupa pi lupa tomo 🚪.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8910727  

pan 🍞 li kama tan pan 🌾.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8910730  

jan seme li toki e ni tawa jan 🧑‍🎤 ni pi kalama musi uta 🎵: o kepeken ilo lukin 🥽 ni?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8910742  

telo lili li awen kin lon anpa pi poki 🍼.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8910768  

sina lukin ala lukin kin e ijo 🎞 ni lon ilo 📺 pi sitelen tawa?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8910779  

jan Ton sama li pali e telo kili 🍊 ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8910782  

telo sewi 🌧 wawa li kama anpa la tomo tawa linja 🚂 li tawa ala.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8910785  

tomo sewi ⛪ pi sin ala li lon ma tomo 🏙 ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8910797  

suwi 🍯 li moli e pipi 🦟.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8910806  

sina pana e ijo 🧠 sina la sina pana e lon tawa jan.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8910808  

kala 🐟 pi luka wan li lon poki 🥫 ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8910840  

ona mute li alasa e mani 💶 tawa tomo sewi ⛪.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8910843  

mi moku ala e soweli 🐷 e soweli 🐮 e sike 🥚 kin.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8910868  

poki wawa 🔋 pi ilo toki 📱 li ala.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8910871  

sina wile ala wile kama jo e moku 🥩 pi sin mute?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8910872  

jan 🧑‍🦽 li kepeken ilo tawa pi supa monsi la ona 🧑‍🦽 li wile jo e lipu ken anu seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8910874  

soweli 🐎 li soweli pona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8910893  

tenpo ale la ona li kepeken kon nasa 🚬.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8910902  

ilo musi 🎺 li ilo musi pi kama musi 🎶.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8910905  

telo pimeja pi ilo sitelen 🖋 li ala.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8910908  

mi kepeken ala len noka 🧦.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8911007  

ona li suli tawa ona 👩.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8911046  

mi wile kama jo e ilo nasa 💊 ni lon esun.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8911051  

o pona e kili 🥔 kepeken ilo 🔪.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8911064  

jan li weka e tomo tawa linja 🚂 ni. tenpo ni la ona li kepeken tomo tawa linja 🚄 ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8911072  

tenpo pimeja open pini ni la mi moku e kiwen kala 🦪 la ni li kama ike tawa mi 🤢.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8911147  

jan lili 👶 li awen ala lon jan lili 👶.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8911228  

tenpo mute la pan linja 🍜 li kama tan ko walo pi kasi pan 🌾.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8911303  

mi moli la mi wile tawa poki ⚱.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8920910  

sina pilin seli la o kepeken ala len 🧥.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8922606  

soweli 🐘 li utala la tenpo ale la kasi lili 🌱 li kama moli.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8923181  

sina pini e sike lukin la mi olin 😙 e sina kepeken uta 💋.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8923279  

jan li pana e sin 🧂 tawa akesi 🐌 la akesi 🐌 li kama moli anu seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8924768  

sina kute ala kute e ni? ni li jan Mali. ona li pilin 😠 la ona li pini e lupa tomo sama ni: lupa tomo li kalama wawa mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8925413  

tenpo wawa la sina o lawa e ni: tomo tawa 🚑 li kama.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8925526  

tomo tawa 🚙 pi mama mama mije 👴 li kepeken telo wawa ⛽ mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8925563  

mi wile tawa tomo sewi ⛪ poka jan Ton.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8925582  

jaki pi soweli 🐑 li pana e pona tawa ma ⛳ pi kasi lili.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8925616  

sina ken ala kepeken monsi lon ni. ni li supa monsi 🛋 pi mama mama mije 👴.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8925641  

jan Ton li kepeken jaki 💩 pi soweli 🐃 tawa kasi lili 🌱.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8925658  

tomo tawa 🚑 li tawa e jan Mawi lon tomo 🏥 kepeken ilo kalama 🚨 laso.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8925699  

soweli 🐏 li ike sijelo lon kulupu la ona li ken pana e ike sijelo tawa kulupu ale ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8926971  

o pali e lupa 🕳 pi anpa mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8927120  

mi moku ala e telo ☕ e telo 🍵 kin.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8927132  

jan Ton li moli e kili 🍄 kepeken ilo 🔫.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8927133  

jan Ton li lukin e pipi 🦋 moli ni: kon li seli e ona 🦋.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8927139  

akesi 🦠 pi lili mute li kama tawa jan Ton.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8927143  

jan Ton li lukin jo e poki 🛄 pi mani 💴 mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8927144  

jan 🧗 li tawa sewi kin nena suli ⛰ kepeken luka 🖐.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8927155  

len 🧥 ni li mani mute. taso ona 🧥 li mani pona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8927157  

soweli 🐩 ni li jo pi jan seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8927158  

o pini e lupa suno tan ni: pipi 🦟 li kama lon tomo.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8929457  

tenpo ni la pan suwi 🎂 li kama pona mute. kon pi pan suwi li kama pona kin!  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8929476  

mi mute li wile kama jo e lipu 🧻 lon esun. tenpo lili la ona 🧻 li ala lon ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8929483  

sina wile tawa li awen pi mute sin lon kon sin. sina lukin kin sama moku 🧀.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8929495  

poki 🛄 ni li kiwen mute kin!  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8929501  

mi wile tawa tomo telo 🚽.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8929531  

jan jaki ni li open e uta tawa pilin 😆 tan mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8929744  

jan Ton li weka ike e ilo 🌂 mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8929747  

sina kepeken ilo 🎤 la o lawa e ona 🎤 poka suli pi mute sin tan uta.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8929770  

meli lili 👧🧑👩 li utala sin e ona sama.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8929805  

mi wile mute e moku 🍟!  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8929823  

jan Ton li pilin 😨 mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8931233  

jan Mali li pilin 🤢 tan ni: ona li moku pi mute ike.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8931253  

jan Mali li pakala e luka pi ona sama kepeken ilo 🔪.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8931614  

o kepeken ilo musi 🎲.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8931633  

jan Mali li kama sona e kule tawa luka 💅 ona lon esun.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8931637  

jan Mawi li tawa tomo telo 🚽.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8931654  

sina pilin 😰! - "mi pilin 😰 ala kin!"  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8931658  

o kepeken 👂. mi wile toki e ijo!  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8931687  

sina 🥴 nasa sin tan telo nasa 🍺 sama soweli 🐷.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8931705  

ijo li kama ike la tenpo ale la sina pilin 😠!  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8931725  

jan Ton o tawa e pan 🍞 tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8931726  

o pana e ilo 🎤 tawa ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8933366  

mi wile kepeken 🚽!  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8933367  

telo 🍼 pi soweli 🐮 li pona tawa soweli 😺 ale.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8933368  

lape li pona tawa soweli 🐈.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8933389  

ilo musi 🥎 li pona mute tawa soweli 🐈 mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8933390  

pipi 🐜🦗🐛 li pona ala pona tawa sina?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8933394  

ona li kute e nimi Akesi 🐍 la ona li pilin 😰.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8933403  

waso 🐔🐤 li kama moli tan soweli 🦊 ni: ona 🦊 li alasa e ona 🐤🐔.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8933409  

soweli 🐺 li alasa e soweli 🦌.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8933422  

kalama 💥 wawa li lawa e ni: soweli 🐶 li mu.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8933425  

soweli 🐿️ li moku e pan 🌾 e kili 🌰 e pipi 🐞 e kili 🍄.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8933446  

mi pana sona e ni tawa soweli 🐶 mi: tenpo suno open 🌄 la ona 🐶 li tawa e lipu 🗞 tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8933465  

sina wile ala wile kute e ni: waso 🦜 li toki?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8933508  

jan Ton li alasa e soweli 🐭 la ona li jo e ona 🐭.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8933521  

ona 👧 li pana e moku tawa waso 🦜 anu seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8933527  

kala 🦐 li pona mute tawa soweli 😸 mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8933534  

soweli 🐻 li ken tawa sewi lon kasi suli 🌲.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8933545  

mi lukin e waso 🦆 ni: luka waso ona li pakala mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8933549  

jan pona mi li pana e kili 🍓 tawa soweli 🐶 ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8933561  

mi sona ala e ni: toki ni li sitelen e seme. taso mi pilin 😦.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8933595  

lawa pi kasi kule 🌹 mi li kama ala lon anpa kiwen ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8933613  

mi lawa e soweli 🐴 mi tawa telo.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8980865  

mi jo e tomo tawa 🚗 e ilo 🏍 e ilo 🚲 tu.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8980884  

ma 🛫 pi tomo tawa kon li pini. taso jan ala li sona tan seme.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8980887  

tu wan 3️⃣ en ➕ luka 5️⃣ sama = luka tu wan 8️⃣.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8980901  

o pana e ilo kalama 🎤 tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8980902  

jan pi ma 🇫🇷 li pona ala tawa jan pona pi ma 🇩🇪. taso telo nasa Win 🍷 pi ma 🇫🇷 li pona mute tawa jan pona pi ma 🇩🇪.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8982213  

mi wile pona e toki 🇮🇹 mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8982348  

tenpo mute la jan Ton li kepeken tenpo moku pi tenpo suno insa lon tenpo ilo 🕝.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8982385  

o lawa ala e ni: jan 👮‍♀️👮‍♂️ li kama.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8982534  

soweli 🦁 li lon ma Apika.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8984255  

tenpo ilo 🕕 li lon tenpo suno open la o pini e lape mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8984270  

kon tawa 🌬 li pakala e len sitelen 🏴.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8984309  

ona 🤑 li wile kama jo e mani 💰 la ona li sama akesi 🦖.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8984324  

ona li moku e telo nasa mute la ona li sama akesi 🦖.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8984436  

ona li suli sama ni: suli pi sike waso 🥚 li lon.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8985617  

tenpo pimeja pini ni la ilo kon 🌡 pi seli en lete li tawa anpa lon nanpa Ala 0️⃣.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8985636  

ona li sitelen e lipu nimi kepeken toki Inli 🇬🇧 la ona li pakala pi lili mute taso.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8985643  

soweli lili 🐀 li tawa insa lon nasin.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8985650  

ona li pali e ni lon ma kasi: kili 🍅 li kama.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8985652  

mi wile ala pana e mani 💶 tawa supa 🛋 ni lon esun.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8985656  

ona 👩 li toki e ni tawa ona 👨: ona 👨 li toki lon ala tawa ona 👩.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8985665  

mute seme pi lipu 📚 ni lon supa?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8985685  

jan 👮‍♀️👮‍♂️ li toki e ni: jan Ton li jan sewi 🦸.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8985694  

mi pali la mi wile kepeken len linja 👔 anu seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8985701  

jan Ton li pana e mani tan ni: pilin pi jan Mali la ona li wile kama jo e ilo 📷 lon esun.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8985727  

jaki 💩 li lon poki esun 🛍 la jan lili 👧👦 li wile ala moku e ijo.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8985731  

ona 🧔 li toki suli e ni: toki ona 👱‍♀️ li toki lon.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8985759  

sinpin pi tenpo lete la lipu kasi 🍂 li kama kule loje.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8985762  

mi mute li weka e ilo 🌂 pi mi mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8985764  

soweli 🐑 li mu seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8985781  

telo sewi 🌧 li kama sin kin.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8985799  

mi pilin e ni: ilo open 🔑 ni li pona ala.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8985816  

suno 🌄 li tawa anpa lon monsi pi nena lili.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8986151  

pona nasa li pana e pona tawa jan wawa 🏋.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8986230  

tawa pi ilo musi 🎠 li kepeken e tawa sama pi ilo tenpo 🕛.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8986253  

ona 👧 li wile e ni: ona 👨‍🎤 li kalama musi uta e ijo tawa ona 👧.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8986272  

tenpo mute la jan Ton li lawa e tomo tawa 🚗 ona lon ma 🅿 ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8986289  

ona li toki e ni tawa mi: mi wile ala wile tawa musi poka ona 💃 kepeken kalama musi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8987060  

akesi linja 🐍 li pakala e noka 🦵 mi kepeken uta 👄.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8987061  

mi pana e mani kepeken ijo mani 💳.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8987082  

mi wile pana e mani Tola 💵 pi luka luka 🔟 tawa sina.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8987083  

ona li open 💥 e utala ona kepeken ilo utala 🔫.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8987086  

nasin ilo ⚙ li ijo pona. taso tenpo mute la ilo ⛓ li kama pakala. tenpo lili la supa noka li pakala.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8987092  

jan Kanipale 🧟 li moku e jan kepeken ilo moku 🍴 la ni li nasin pi tenpo kama anu seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8987098  

akesi sewi pona 👼 li seme? ona li waso jan tawa jan Kanipale 🧟.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8987103  

insa lawa 🧠 ona 👴 li pona ala la pilin ona 👴 la ona 👴 li sitelen sijelo kiwen 🗽.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8987106  

nunpa 0️⃣ li pona tawa jan lawa pi tomo tawa telo. ona li pilin e ni: nanpa 0️⃣ li sama e ilo sike lon telo suli.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8987116  

nunpa 0️⃣ li pona tawa jan lawa pi tomo tawa telo. ona li pilin e ni: nanpa 0️⃣ li sama e ilo sike lon telo suli.  
- likely misspelled word: nunpa  
- http://tatoeba.org/eng/sentences/show/8987116  

tenpo pmeja pini ni la mi moku e pan en sin Kuli 🍛.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8987363  

tenpo pmeja pini ni la mi moku e pan en sin Kuli 🍛.  
- likely misspelled word: pmeja  
- http://tatoeba.org/eng/sentences/show/8987363  

ijo ni li ni la o kute e toki lawa pi jan 🧑‍⚕️ sina.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8987383  

jan Ton li pali e ni: pipi 🐝 li kama suli.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8987395  

ona mute li musi kepeken lipu musi ♣♦♥♠.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8987417  

soweli 🐑 mute pi jan 👨‍🌾 li moli tan soweli 🐶 utala.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8987426  

soweli 🐶 utala li moli e soweli 🐑 mute pi jan 👨‍🌾.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8987427  

mi pini e sike lukin 👁 mi. mi kute 👂 e kalama 🎵 pi telo suli 🌊.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8987445  

sina o open e lupa tomo 🚪.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8987447  

tenpo pini la ona li jan 😊 ni. tenpo ni la ona li jan 😶 ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8987454  

jan Ton li kepeken sin kin e ilo lukin 👓 anu seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8990848  

jan Ton li pana e sike lili nasa 💊 lon poki jaki 🚽 kepeken telo.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8990910  

mi 😴 lape lon tomo tawa telo 🛳.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8990951  

nasin musi pona pi kulupu jan li kama tawa tenpo ni: jan li moku ala e jan la taso kin jan li moku ala e soweli e waso e kala ...🐸🦗.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8994269  

ike ale li ken lon walo uta 🦷 pi wan taso.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8994404  

jan 🙂 li jan 😇 ala. jan 🙂 li jan 🤑 ni li sama jan 🤑 ante. ona 🤑 ale li wile kama jo e mani 💸 taso.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8994408  

jan pona 😇 li wile pi mute ale li pana e ijo tawa jan ante kin.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8994423  

a! jan 👿 mute li ike mute tan ni: ona 👿 mute li ken lukin e ni la ona 👿 mute li olin e ni: jan 😠 li utala e jan 😭 kepeken luka kiwen 👊 la ni li musi mute tawa ona 😈 mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8994449  

sina wile lukin e pilin insa pi jan ante. taso sina ken lukin taso e len 👔 🦺 ona. taso pilin insa pi jan ante li kama ala tawa lukin sina.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8994506  

o pali mute 😓. o pilin pona 😊 kin. tan ni la pilin 😤 utala li kama kin.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8994559  

🦷 walo uta li utala e jan 😴 pi lape mute la sike lukin ona 🥺 li kama suli mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8994590  

jan 🥳 ni li kepeken mute unpa e telo nasa la tenpo mute la jan ante 😠 li toki utala ☠ tawa jan 🥳 ni. tan ni la ona 😠 li wile e sama kin.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8994653  

ona li jan lili 👶 la ona li kama sona e ni: ona li kalama uta la moku li kama tawa ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8995838  

sina 😠 pilin utala la o kepeken nanpa tawa ... tu tu. ni li pona ala la o pilin utala mute 🤯.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8995847  

sitelen sona li kama ala la ni li sama. jan li wile sitelten la jan li ken.  
- likely misspelled word: sitelten  
- http://tatoeba.org/eng/sentences/show/8995854  

ona mute li pilin 😨 ala tan utala.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8995883  

telo li pona mute tawa soweli 🦛.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8995889  

mi lukin e ni lon poka suli: seli 🔥 li kama open.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8995891  

tenpo ilo 🕕 la jan Mawi li pini e lape ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8995897  

tenpo ilo 🕕 la jan Mali li kama tan lape.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8995898  

kasi kule 🌸 li nimi pi pona mute ale tan ma.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8996370  

ona li open e ilo 📺 pi sitelen tawa.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8996455  

mi tawa tan ni: mi kepeken kon nasa 🚬.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8996457  

o pana kin e telo nasa 🍺 tawa jan Ton kepeken mani.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8998201  

ijo li kama suli pi mute en mute la sike lukin pi jan 😲 ale li kama suli. nasa li kama suli la taso kin.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/8998839  

jan Mali li seli e moku la ona li kepeken sin 🌶 pi mute ike.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9001858  

moku li sin 🌶 la ni li pona ala pona tawa sina?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9001866  

moku li sin 🌶 la ni li pona ala pona tawa sina mute?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9001870  

moku pi sin 🌶 li pona mute tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9001872  

moku pi sin 🌶 li pona tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9001874  

moku pi sin 🌶 li pona ala tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9001892  

moku 🍛 ni li sin 🌶 pi mute ike tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9001918  

moku li sin 🌶 la ni li pona ala tawa jan Ton.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9001931  

kili 🌶 ni li sama seli 🔥.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9001939  

moku telo ni li sin 🌶 mute kin.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9001944  

moku telo ni li sin 🌶 pi mute ike.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9001953  

moku ni li sin 🌶 pi mute ike.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9001958  

moku li sin 🌶 la ni pona tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9001961  

nena sinpin pi soweli 🐶 ni li wawa.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9001981  

jan li pilin e moku ko 🧀 ni kepeken uta la ona 🧀 li sin 🌶.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9001999  

soweli 🦛 li ken utala e jan.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9002404  

jan li moku e telo nasa Win 🍷 la ona li ken lukin e ijo pi tenpo kama.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9002419  

mi jan ni: ona li seli e pan 🥨🥖🥐.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9002450  

o seli e suwi pan 🥮.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9002454  

o pona e tomo mi kepeken ilo 🧹.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9002459  

jan Mawi li wile pona e tomo kepeken ilo 🧹.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9002473  

mi mute li pona kepeken ilo 🧹.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9002476  

o weka ala e ni lon sona sina: o pona e tomo moku kepeken ilo 🧹.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9002481  

o pini e lupa tomo kepeken ilo 🗝.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9002503  

soweli 🦮 li pana e pona tawa jan 🧑‍🦯 pi lukin ala.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9002517  

suno pi sike luka 💍 ni li weka.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9002571  

suno lon selo pi soweli 🐈‍ mi mute li weka.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9002583  

mi pali e len kepeken palisa lili en linja 🧶 la ni li pona tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9002596  

ona li kepeken palisa lili en linja 🧶 la ona li pakala e ona sama.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9002600  

mi moli e waso 🐔.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9002605  

ko jelo moku 🧈 li ko pilin.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9003484  

noka mi li kama sama ko jelo 🧈.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9003493  

o pona e walo uta 🦷 sina kepeken ilo.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9003759  

sike lili nasa 💊 ni li kama lon pilin 💓.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9003855  

tomo tawa linja 🚋 li tawa kepeken wawa ilo ⚡.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9005799  

ilo tenpo 🕰 ni li kepeken wawa ilo 🔌.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9005804  

ilo ni li kepeken wawa ilo 🔌.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9005808  

tomo tawa 🚘 li tawa kepeken wawa ijo 🔌.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9005810  

pimeja ale pi sike ma 🌏 li ken ala weka e suno pi ilo sona 🕯 wan.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9005829  

mi lon poki 🛀 kepeken telo seli la mi kama lon pilin pona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9005856  

jan li pilin suwi lon tomo 🏨 ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9005889  

lipu kasi 🍂 li kama anpa.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9005944  

selo 🎭 pi sitelen lawa li kama weka.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9005958  

o kama sona wawa e toki pi ma Nijon 🇯🇵.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9005998  

telo nasa Win 🍷 li pana e pona tawa ni: moku li kama pona lon insa sijelo.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9007887  

telo nasa Win 🍷 li pana e pona tawa lon insa sijelo.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9007888  

tenpo ale la jan li seli ala e moku 🍆🥔🌽 la ni li pona ala tawa insa sijelo jan.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9007889  

mi pana kin e telo 🍹 tawa jan mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9007895  

jan lawa 🤴 li toki e ni: o pana e ijo 🎖 tawa jan mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9007896  

jan moku pi telo nasa mute li kiwen e ona sama lon palisa pi poki 🥤 ona kepeken luka ale.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9007926  

pipi 🦟 li tawa lon kon la kalama li lon.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9007951  

soweli 🦡 li pali e lupa pi anpa mute lon ma.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9008086  

mi seli e kala 🐟.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9008090  

mi seli e waso 🦆.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9008093  

tenpo kama la mi seli e moku soweli 🥩.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9008094  

moku soweli 🥩 li lete la ni li pona tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9008096  

o seli e lili tu tu pi kili 🧄.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9008098  

ona li seli e moku waso 🍗 pi pona mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9008099  

lukin ona 👩‍🦰 li tawa lon lipu 💌 ona 👨‍🦱.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9008162  

loje uta 👅 pi jan Ton li tawa lon nena kute 👂 pi jan Mali.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9008167  

ona li pana e mani 💴 lon poki 👜 mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9008169  

o pakala ala kepeken kiwen 💅 pi palisa luka.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9008174  

pipi 🦟 li pakala e sina la o tawa ala lon nena lili kepeken kiwen 💅 pi palisa luka.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9008176  

pona sijelo pi jan Ton li sama pona sijelo pi jan 👴 tenpo suli.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9010237  

jan li lukin e lipu ni la ona 📘 li sama mani.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9010265  

ilo sona 💻 li kepeken nanpa.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9010310  

mi ken ala ken olin e soweli 😺 sina kepeken luka?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9010377  

sina wile la sina ken olin e soweli 🐕 kepeken luka.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9010389  

o weka ala e ni lon sona sina: o olin e soweli 🐈. o telo e kasi suli 🌳.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9010403  

soweli 🐪 li ken ala lukin e nena ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9010410  

soweli Kamelo li jo e nena wan 🐪 anu nena tu 🐫.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9010413  

mi tawa e poki 🍵 wan pi kasi seli tawa sina.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9010421  

poki 🥫 ni li pona tawa ni: tenpo suli la moku li pona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9012308  

uta 👄 pi akesi 🐲 ni li pana e seli 🔥.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9012309  

seli 🔥 li kama tan uta 👄 pi akesi 🐲 ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9012311  

akesi 🐉 li jo e luka waso. uta 👄 ona 🐉 li pana e seli 🔥.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9012321  

lipu 📕 ni li toki e ni: akesi 🐉 ni li ken ala kepeken seli uta.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9012337  

lipu kasi 🍁 li kama suli lon palisa lili pi kasi suli 🌳.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9012360  

kili pi kasi suli kili 🍊 li kama suli lon ma seli.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9012364  

kasi 🌱 li kama suli tawa suno 🌞.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9012366  

kasi 🌱 li kama suli la ona 🌱 li wile kepeken suno.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9012376  

kasi suli kili 🌳🍒 li kama suli lon poka pi nasin 🛣 ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9012383  

mi mute 👷‍♂️👷‍♂️ li pali e tomo sina sama wile sina.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9012446  

jan Ton li pini e lape pi jan Mawi lon tenpo ilo 🕡 sama wile ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9012480  

mi ken ala ken kama jo e jan utala 🧟🧟 lon tenpo mani?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9012532  

o lili e kalama pi poki toki 📻.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9012936  

o lili e kalama pi ilo sitelen tawa 📺.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9012942  

o tawa weka e poki 📦 ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9012952  

o tawa e poki 📦 lon anpa.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9012956  

o tawa pi mute sin e supa monsi 🪑 lon sewi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9012967  

pona o tawa weka e supa monsi 🛋 ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9012995  

pona. o tawa weka e supa mons 🪑 ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9012996  

pona. o tawa weka e supa mons 🪑 ni.  
- likely misspelled word: mons  
- http://tatoeba.org/eng/sentences/show/9012996  

ilo tenpo ⏱ li kepeken nanpa.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9013000  

tu tu 4️⃣ li weka tan tu tu 4️⃣ li sama e ala 0️⃣.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9013012  

o tawa e telo nasa 🍺 tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9014351  

o pana e suwi 🍭 tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9014473  

mi tawa e ilo lukin 👓 mi lon ma ona 👓.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9014814  

ona li pana e pona ni: ilo tenpo ⌚ ni li jo mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9016196  

ona li pana e lipu 💌 tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9016201  

mi wile e tomo tawa 🚕.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9016220  

tomo tawa telo 🛳 li ken kama anpa.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9016399  

tenpo lili la suno 🌞 li kama anpa.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9016401  

mi lukin e ni: tomo tawa telo ⛴ li kama anpa.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9016404  

sike ma 🌏 li ken kin kama moli anu seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9016406  

utala Aton⚛ li kama la jan ale li kama moli anu seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9016419  

kon pi ilo tawa 🚲 mi li weka tan sike.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9016431  

ona li pali e ni: len lawa 🎩 mi li lukin sama supa.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9016437  

jan 🧑‍🚀 li tawa lon sewi la ona li kepeken len.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9016481  

kasi suli kili 🌳🍎 li jo e lawa kasi kule 🌸 pi pona lukin.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9016491  

lawa kasi kule 🌸 li pona lukin lon kasi suli 🌳 kili 🍎.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9016497  

selo noka 👠 mi li lili pi mute ike.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9016513  

ilo toki ☎ li pini.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9016950  

ilo 🤖 li kama pakala la jan li ken ala lawa e ona 🤖.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9016993  

ona li pali ala e ijo. taso ona li lukin e lipu 📰.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9017012  

jan ala li ken lawa e ilo 🤖🤖!  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9017074  

jan Ton li pilin lape la uta ona 🥱 li kama suli lon nasin sama supa.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9017091  

mi lon tomo 📽 la mi kepeken supa monsi 🪑 lon sinpin mute la ni li pona ala tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9019089  

meli li open e lupa suno la ona li ken lukin e ma kasi suli 🌲🌲 e ma kasi kule 🌻🌷 lon poka lili mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9019136  

mi wile kepeken ilo 🪒 tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9019141  

sina kepeken ilo 🪒 tawa noka sina anu seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9019143  

ilo 🪒🪒 li lon seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9019144  

jan Ton li kepeken ilo 🪒 la telo loje ona li kama.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9019147  

tenpo mute seme la sina kepeken ilo 🪒 tawa sina?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9019149  

ilo 🪒 ni li sin mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9019151  

mi weka e linja lawa anpa mi kepeken ilo 🪒.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9019153  

mi kepeken ala ilo 🪒 tawa noka mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9019154  

ona li kepeken ilo 🪒 la nena kute ona li kama telo loje.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9019157  

mama mama mije 👴 la o sina pali e seme? - mi kepeken ilo 🪒 tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9019160  

ona li open lon tenpo 🕣.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9019683  

ko lete ❄ li lon nena 🏔.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9019684  

ona li lukin sama ni: jan li kepeken nasin 🛣 ni la jan li kepeken tenpo mute tan lon ni tawa ma tomo 🏙.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9019693  

jan Ton li pali e ale tawa ni: pali ona li kama pini lon tenpo 🕝.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9021889  

tan seme la sina mute li kama jo e akesi 🐢 lon esun?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9021897  

ona li moku e telo kili la ona li kepeken palisa tawa poki 🥤 ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9021906  

ilo nasa 💊 li kama lon pona mute tawa pona sijelo ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9024360  

ma tomo lili en ma tomo 🏙 pi mi mute li wan kepeken nasin sewi 🌉.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9024364  

ona 👩 li ken musi lon ilo musi 🎹 kepeken nasin kalama 🎼 ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9024373  

ko lete ❄ wan li kama lon nena sinpin 👃 pi jan Mawi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9024384  

ona li pilin wawa ala tawa ni: ona li ken toki e ni tawa mama ona: ona ♂ li olin 👬 e mije.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9024409  

sina ken ala ken pana e sin walo 🧂 tawa mi?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9024421  

sina weka e sin walo 🧂 lon anpa la pona nasa li ante la ike li kama!  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9024438  

o pilin suwi kepeken supa 🛋.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9024465  

mi lawa e soweli 🐐🐐 ni tawa nena ⛰.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9024836  

jan Mali taso li kepeken len 👗. meli ante li kepeken ala e ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9024837  

poki 🛁 anu ilo 🚿 la poki 🛁 li pona tawa ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9024858  

tenpo seme la tomo tawa kon 🛬 li kama lon ma tomo Tokio?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9024860  

kon sewi li telo la ni li pona tawa kasi lili. ona 🌱 li kama suli.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9024871  

tomo tawa linja 🚆 li tawa lon nasin linja seme 🛤 tawa ma tomo Lonon?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9027194  

tenpo pini la sina pana e ilo tawa 🚲 sina lon seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9027207  

lipu 📘 ni li pana e pona ni: tenpo kama la sitelen sina li pona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9027230  

kon li seli lon ma la telo nasa 🍺 li kama tan poki 🛢🍺 la ona 🍺 li pona mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9027234  

mi pilin e ni: ilo tawa 🏍 ni li jo pi jan Ton.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9027242  

ona 👨 li kama lon tomo la ona 👩‍🦱 li lukin e ni: ona 👨 li pilin lape mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9027573  

o kepeken ilo nasa 💉 ni la sina kama pilin pona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9027579  

telo sama pi kili 🍋 li mute lon kili 🍍.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9027618  

jan 👮‍♂️👮‍♀️ li wile e sona sina la ona 👮‍♂️ li toki ala toki kin tawa sina?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9027636  

nimi pi kasi kule 🌹🌹 ni li lon poki li nimi Lose.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9027654  

soweli 🐇 anu soweli 🐿 la seme li suwi pi mute ale?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9027671  

ona 🏇 li tawa telo suli.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9027682  

ona li tawa telo suli 🏖 kepeken soweli 🐎.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9027690  

tenpo kama la mi mute li tawa e poki🛢🍺 tawa kulupu musi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9027708  

mi jan ni ala: ona ♀ li sama jan ni lon tenpo pini.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9027725  

anpa pi lupa 🕳 ni li poka suli seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9027730  

mi kama jo e telo pimeja ☕ la ni li pona tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9027741  

mi mute li awen e jo ni pi tu wan tan ma Eten: suno lili 🌟 e kasi kule 💮 e jan lili 🧒.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9029273  

mi kepeken ala selo noka 👟 la mi pilin e ko lon anpa pi noka mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9029562  

o lukin e ni: ken la ona li lon poki 👜 sina!  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9029571  

sina pana ala e suwi lon telo pimeja ☕ sina.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9029580  

sina kama poka lili sin la mi kepeken ilo utala 🔫!  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9029585  

ilo tenpo ⌚ sina li toki e seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9029587  

kon tawa 🌬 li tawa weka e ona. taso mi pali e ni kepeken ilo 🔨: tenpo ni la ona li ken ala kama weka.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9030974  

sitelen lukin 🖼 mute li lon tomo esun ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9030979  

ona li pilin 😨 tan ni: tenpo kama la ken la ona li wile lon tomo 🏥 pi pona sijelo lon tenpo 🎄.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9030987  

jan tu li kepeken ilo tawa 🚲 wan la pakala li ken kama tawa jan tu.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9030994  

telo walo 🥛 li kama tan soweli 🐮.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9031003  

kili 🍓 li pona ala pona tawa sina? - "lon. ona li pona mute tawa mi."  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9031005  

meli lili 👧 li selo e ona sama lon tomo 🏠 pi jan poka 👩‍🦰.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9031008  

mi kepeken ala ilo 📱 tawa ni: tomo tawa 🚑 li wile kama.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9031021  

tenpo ale la ona li kama tan tomo ona lon tenpo 🕖.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9031474  

tenpo 🕕 la ona li kama lon ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9031486  

jan li kepeken tomo 🏠 ni la jan li pilin suwi mute lon ona 🏠.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9031503  

mi lukin e lipu 📰 suli.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9031520  

ko tawa wawa 🌪 li pakala e ijo mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9033091  

jan li ken ala ken tawa e ko 🧀 lon poki kepeken lete mute?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9033250  

mi wile e ni: jan li pana e mi lon poki ⚰ kepeken lete mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9033253  

sina kama jo sin e pan 🥮 kepeken luka anu seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9033310  

mi wile kin poki e len telo 👙.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9033321  

selo noka 🥾 ni li pona ala.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9033330  

kasi kule ale 🌻🌹🌷 ala li pona tawa kulupu 💐 pi kasi kule.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9033339  

sina wile pali e kulupu 💐 pi kasi kule la sina ken ala kepeken kasi kule ale 🌻🌹🌷.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9033345  

jan pona ni li pona. taso jan pona ante li pona ala tawa tenpo 🍵 ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9033356  

waso tu 🐓🐓 li lon nena jaki la ni li pona ala.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9033363  

mi ken kepeken lipu mani 💳 seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9033375  

lipu 📄⚕ ni pi ilo nasa li pona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9033399  

sina kepeken lipu 📙 ni la ken la ona 📙 li pona mute tawa sina.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9033430  

sina moku e kili seli 🍟 la ona 🍟 li pona ala tawa pona sijelo sina.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9033437  

jan li kepeken supa 🪑 ni la ni li pilin suwi ala.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9035662  

jan li lon tomo ☕ ni la ni li pilin suwi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9035677  

jan tawa li kama lon ma Sasali 🇩🇿 la ni li pona tawa kulupu ma 🇩🇿.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9035805  

tenpo ale la tenpo lete li lon nena suli 🏔.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9035817  

tenpo pimeja li kama lete lon nena 🏔.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9035820  

jan li sona ala e ni: ona sama li kepeken selo lawa 🎭 la ona li kepeken pi pona mute ale e selo 🎭.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9035855  

ona 👩‍🦰 li pana e ilo 🗝 lon poki 👜 ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9035863  

ni li lon kin e insa pi soweli 🐩.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9035867  

mi kepeken mani 💳.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9035873  

jan Ton li tawa weka ala kepeken noka wawa la mi lawa e ni: jan 👮‍♀️👮‍♂️ li kama tawa ona!  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9035886  

soweli 🐶 li kepeken nena 👃 lon poki 🛄 ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9035891  

tenpo suno ale la sina kepeken ilo 📺 anu seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9035894  

ilo 🚦 li kama loje.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9035931  

suwi pi mute ike li lon poki ☕ pi telo pimeja.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9035938  

tenpo suno open kama la mi tawa lon tenpo 🕖.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9035960  

mi sona e ni tan lipu 📰 taso.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9035978  

jan lili ale li kama jo e suwi 🍬 tu wan tan mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9035983  

mi wile e ni: tenpo 🕜 la mi lon ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9035988  

ona 👑 li ken lawa e ni la jan li wile kepeken toki pi wile ona 👑 la ona 👑 li ken lawa e sitelen sona pi jan ale.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9036102  

tenpo suli la o kepeken lon sina lon utala ala. 🖖  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9037560  

jan 👨‍⚕️🦷 li toki e ni tawa mi: o tawa ala.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9037677  

soweli 🐎 li weka e jan Mali tan monsi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9037834  

o tawa e sike 🏀 tawa mi lon kon.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9037835  

tenpo pi sinpin ike la waso 🐓 li mu la jan li moli e ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9037850  

jan lili li toki utala li wile ala e ni: ona li wile moku e ilo nasa 💊.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9037853  

jan 👮 li toki tawa meli lili 👧 li wile e sona ni: tomo tawa ni li jo pi sina mute anu seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9037859  

monsi pi tenpo pimeja 🕛 la ona 👩 li kama tawa lupa 🕳 pi poki moli⚰. ona 🕳⚰ li open. ona 👩 li pilin 😨 mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9037863  

tenpo wan la soweli lili 🐭 li moku e ko 🧀 la ona li kama sin.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9037903  

jan Mawi li moku e telo 🥤🍋. jan Ton li moku e telo 🍺.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9037911  

tenpo seme la sina kama tawa ma Nijon 🇯🇵 lon tenpo pini?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9037918  

tenpo 🕡 la o kepeken ilo ☎ tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9037940  

len 🧥🧥 li lon poki len pi mi mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9037947  

sin 🧂 walo li mute pona la o pana ala e sin 🧂 pi mute ike tawa moku 🥣.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9037976  

sina lukin e lipu 📚 pi mute seme lon tenpo sike pini ni?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9037984  

tenpo ni la ilo 🔔 li kalama.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9039777  

jan Ton li pilin 😨 tan akesi 👻.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9039779  

akesi 👺👺 li kama tawa tomo pi jan Ton.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9039781  

jan 🤡 li pona ala tawa jan Mali 😧.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9039783  

jan Ton li kepeken len pi mije 🎅.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9039784  

jan Ton li kepeken len pi meli 🤶.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9039785  

tomo musi 🎪 pi ma Pokasi 🇧🇬 li pona ala pona tawa sina?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9039789  

lon. mi pali lon tomo musi 🎪.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9039790  

ona li tawa kepeken supa sike 🛹.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9039792  

mi kepeken supa sike 🛹 la ni li pona tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9039794  

sina ken ala kepeken supa sike 🛹 lon ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9039795  

jan Pia li jan jo pi supa sike 🛹 sin. ona 🛹 li suli mute tawa jan Pia.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9039798  

mi pilin e ni: jan li kepeken supa sike 🛹 la noka 🦵 li pali wawa.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9039799  

ma awen 🚏 pi tomo tawa kulupu 🚍 li lon seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9039802  

tomo wawa ⛽ pi tomo tawa li lon seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9039806  

tomo wawa ⛽ pi tomo tawa li lon ni anu seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9039807  

tomo tawa 🚑 li tawa kepeken ilo kalama 🔊 en ilo suno 🚨 laso.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9039813  

o kepeken ilo pini ⚓ pi tomo tawa telo ⛴.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9039816  

tomo tawa telo ⛴ li kepeken ilo pini ⚓.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9039818  

tenpo kama la mi ken kepeken ilo kon 🪂 la ni li pona tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9039823  

o weka ala e ni lon sona sina: o open e ilo kon 🪂 sina.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9039827  

ona mute li pali e ni: ilo tawa sewi 🚀 li ken open.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9039838  

ilo tawa sewi 🚀 li tawa sike lon mun.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9039840  

ilo tawa sewi 🚀 li tawa open lon ma sewi 🌌.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9039846  

mi lukin e tomo tawa sewi 🛸.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9039851  

mun sin 🌚 li lon.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9039856  

tenpo pini la mun sin 🌚 anu mun suli 🌕 la seme li lon?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9039859  

kon 🌡li kama lete.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9039864  

kon 🌡li kama seli.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9039865  

sitelen 🌈 li toki e seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040089  

jan li ken lukin e kule ni pi sike kule sewi 🌈: loje. loje jelo. jelo. laso jelo. laso. laso pimeja en laso loje.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040312  

sina kepeken ala ilo ⛱ la selo sijelo sina li kama loje li kama weka.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040391  

sike sewi ☄ li kama tan lon seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040419  

ona li toki e ni tawa kulupu: jan li lukin e sike sewi Kome ☄ sin.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040434  

tenpo wawa la sike sewi Kome ☄ li tawa sike ma 🌎.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040457  

telo 💧 ale pi lili mute li suli.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040482  

mi kute e kalama ni: telo 💧 pi lili mute li kama anpa.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040595  

telo 💧 en telo 💧 li kama suli la telo suli 🌊 li lon.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040618  

tenpo 🎃 li lon tenpo seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040644  

tenpo 🎃 li pona mute tawa jan lili.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040646  

tenpo 🎃 la sina mute li pali e seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040647  

jan Ton li kepeken selo 👹 pi tenpo 🎃.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040648  

tenpo Siwete 🎆 la sina pali e seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040674  

tenpo musi 🎆 pi tenpo sike sin li kama la sina pali e seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040678  

sike kon 🎈 mi li kama pakala 💥.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040701  

jan ala li pilin e ni: sike kon 🎈 li ken kama pakala 💥.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040702  

tenpo wawa la sike kon 🎈 li kama pakala 💥.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040704  

kon mute li lon sike kon 🎈.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040706  

ona mute li pali e ni: sike kon 🎈 li tawa sewi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040711  

jan Ton li pana e kon lon sike kon 🎈.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040715  

tenpo wawa lili la sike kon 🎈 li kama sewi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040721  

jan Ton li kepeken kon tawa sike kon 🎈.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040725  

kon li insa lon sike kon 🎈.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040728  

sike kon 🎈 li tawa kon lon sewi pi nasin ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040730  

sike kon 🎈 li tawa lon kon.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040737  

sike kon 🎈 li tawa kon lon sewi pi ma tomo.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040739  

jan Ton li kepeken kon Heliun tawa sike kon 🎈.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040743  

jan Ton li pana e sike kon 🎈 tawa jan Mali. ona 🎈 li lukin sama sijelo 🖤.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040748  

sike kule 🎈 li kule li tawa lon sewi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040751  

kon tawa 🌬 li lawa e sike kule 🎈 lon ni en lon ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040755  

ilo musi 🧩 li pona mute tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040762  

ilo musi Puse 🧩 li jo e ijo 🧩 pi nanpa 500.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040768  

jan Ton li kepeken ilo musi Puse 🧩.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040769  

mi kama jo e ilo musi Puse 🧩 sin lon esun.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040775  

jan Ton li toki e ni: lon li sama ilo musi Puse 🧩.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040778  

ona 😎 li kepeken ilo 🕶 tan suno 🌞.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040786  

mi 😎 jo e ilo 🕶 anu ilo 🕶 li jo e mi 😎.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040791  

ona 😎 li kepeken ilo 🕶.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040793  

jan Ton li kepeken len 🥼 pi sin ala.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040795  

jan Ton li open e poki 💼 ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040818  

lon seme la poki 💼 mi li lon?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040819  

poki 💼 mi li lon seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040821  

jan Ton li lukin lon insa pi poki 💼 ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040823  

jan Ton li tawa e poki 💼 pimeja ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040825  

mi wile kama sona e ni: mi lawa e ilo tawa 🚜.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040842  

tenpo ni la ilo tawa 🚜 ni pi sin ala li tawa e jan tawa.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9040844  

ni kin li soweli 😺 mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9042484  

mi kama jo e lipu ✉ pi jan pona mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9042569  

jan pona mi li lawa e lipu ✉ tawa mi kepeken tomo 🏤.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9042579  

jan li open tawa jan ale pi sike ma 🌏 lon lipu Tatojeba la ni li pona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9042602  

tenpo 🎄 la mi mute li awen lon nena suli 🏔.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9042611  

o lukin e ni: mi pali e ale 🍓🥕🍅 ni. ona 🥒🧅🥦 ale li kama tan ma kasi mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9044414  

kon pimeja 💭 li lon sewi la telo sewi 🌧 li kama.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9044418  

nimi *** li sitelen esun pi ma Sonko 🇨🇳.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9044436  

mi awen lon tomo mi kepeken ilo 📺 pi sitelen tawa la ni li pona tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9045170  

ona li sitelen e lipu ✉ kepeken toki Inli anu seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9045177  

lipu 📖 ni li lili. lipu 📘 ante ni li suli.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9047891  

jan 🧑‍⚕️ pi pona sijelo li toki e ni tawa mi: mi moku e sike waso 🥚 la ni li ike tawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9047983  

ona li ken pona kepeken ilo 🎿 pi tenpo lete.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9047994  

kon ni li pona tawa nena sinpin 👃 anu seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9048070  

sina pana ala pana e ilo wawa 🔋 lon tomo esun ni?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9048139  

mi wile e suwi 🍮 kin.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9048153  

jan 🧔 ni li jan pona mi. ona 🧔 li sitelen 🎨 e lipu 🖼 ni: ona 🖼 li pona tawa sina.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9048859  

jan Ton li kepeken monsi 🧘‍♂️ lon anpa ma kepeken nasin Joga.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9048887  

o tawa. mi mute o tawa tomo 🕌.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9048889  

suwi pan 🍰 sina li pona kepeken uta.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9048896  

jan li mama e kili Melon 🍈 sin.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9048901  

kili 🍏🍏 ni li pona ala. ona 🍏 li wile kepeken tenpo pi mute sin.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9048920  

tenpo pimjea la kon tawa 🌬 li tawa wawa.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9048938  

tenpo pimjea la kon tawa 🌬 li tawa wawa.  
- likely misspelled word: pimjea  
- http://tatoeba.org/eng/sentences/show/9048938  

tomo tawa kulupu 🚍 li tawa mi mute tan tomo 🛬 pi tomo tawa kon tawa ma tomo 🏙.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9048952  

ala li lon insa pi poki 🧴 ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9048982  

mi wile e pan 🥐 e ko 🧈 e telo 🍵 pi kasi seli e suwi 🍯.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9048989  

kon pi telo kasi seli 🍵 ni li pona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9051010  

mi lukin e kala 🦞 sin la mi wile moku e ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9051017  

len noko 🧦 pi jan Ton li jo e lupa 🕳.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9051020  

len noko 🧦 pi jan Ton li jo e lupa 🕳.  
- likely misspelled word: noko  
- http://tatoeba.org/eng/sentences/show/9051020  

lupa 🕳 li lon len noka 🧦 pi jan Ton.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9051024  

jan Sesu li pali e ni: telo li kama lon telo nasa Win 🍷.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9051028  

akesi Patelia 🦠 pi lili mute li kama ala mute lon telo nasa 🥃.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9051124  

jan Ton li pilin ike tan ni: uta ona li wile suli e sike musi 🏐 kepeken kon.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9051162  

o suli e sike kon🎈 kepeken kon.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9051173  

jan li tawa lon ma seli 🏜 pi ko mute la tenpo mute la jan li kepeken soweli 🐫.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9053490  

jan Ton li selo e ilo utala 🔫 ona lon anpa pi len 🧥 ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9053499  

kasi li selo e poki pi tomo 🏡 ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9053510  

mama mije mi li alasa e kala kepeken ilo 🎣 la ona li kama jo e kala mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9053518  

o kama jo e suwi pan 🥧.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9053519  

poki 🎒 sina li open!  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9053525  

tan seme la ona li telo e ilo moku 🍽 kepeken luka? ona li jo kin e ilo ni: ona li telo e ilo moku.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9053532  

tenpo pini la jan Ton li tawa lon nasin tawa tomo sewi ⛪.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9053552  

mi wile pana e mani 💵 pi nanpa 300💲 tawa supa sitelen ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9053594  

sitelen len 🏳️‍🌈 pi ona mute li kule mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9056468  

jan Ton li nasa tan ni: ona li moku ala e telo 🍵 pi kasi seli e telo pimeja ☕ e telo nasa 🍺.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9056735  

tenpo 🎄 li kama sin poka lili.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9056816  

kasi lili li kama suli. taso soweli 🐴 li kama moli.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9056874  

mi mute li tawa tomo moku 🍕. taso ona li tawa tomo 📽 pi sitelen tawa.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9056876  

sina wile lon insa pi tomo tawa linja la o pali e ni: lipu 🎫 sina li kama jo e lupa.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9059610  

mi lawa e ni: sina kama jo e lipu 🎟 lon esun.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9059623  

mije lili li toki e ni: ona li pali ala e ike ni: ilo tawa 🚲 li kama weka. taso jan li lukin e pali ike ona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9059632  

sina ken ala ken e ni: waso 🐔 li awen lon luka sina?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9059676  

ona li wile e suwi. taso ona li kepeken sin 🧂 walo.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9059687  

o lukin e soweli 🐈 lon sewi tomo.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9063042  

poka lili la sike musi 🏈 li kama wawa tawa lawa mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9063050  

sina mute li lukin e ale lon lipu 📗 anu seme?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9063055  

tenpo seme la tenpo 🌧 li pini?  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9063065  

ona li wile e ni: ona li kama jo e ilo tawa 🚲 sin lon esun.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9063078  

tomo tawa telo 🛳 li tawa lon poka lete.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9063079  

ona li lon poka suli tan lon pi sike ma 🌎.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9063106  

seli 💥 pi lili mute li kama open la seli 🔥 li kama.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9063141  

seli 💥 pi lili mute li pali e ni: ko li pakala wawa 💥.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9063142  

tenpo mute la seli 💥 pi lili mute li open e seli 🔥 suli .  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9063146  

seli 🔥 wawa li kama tan seli 💥 pi lili mute.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9063153  

kili li pona tawa mi. kili 🍐 kin.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9063504  

kili li pona tawa mi. kili 🍊 kin.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9063506  

o moku e lili taso tan moku 🍔🍕🍟 ni.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9063550  

mi weka e tomo len ⛺ mi. tan ni la kon tawa 🌬 wawa li kama.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9063588  

tan kon tawa 🌬 wawa la mi weka e tomo len ⛺ mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9063590  

jan li tawa lon sike nasin kepeken ilo tawa 🏍 la sinpin la ona li wile e ni: tenpo tawa pi ilo tawa 🏍 li lili.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9063701  

walo uta 🦷 ona li pona taso.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9063998  

tomo tawa kon li kama lon tenpo ilo 🕗 pi luka tu wan.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9067179  

janTon li alasa e kala la ni li pona tawa ona.  
- likely misspelled word: janTon  
- http://tatoeba.org/eng/sentences/show/9067218  

ilo mani li weka e lipu 💳 mi.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9067297  

kala 🦞 li pona.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9067331  

jan Ton li jan sewi. ona li kama jo e sike 🏅.  
- Contains emoji  
- http://tatoeba.org/eng/sentences/show/9071225  

temp ale la jan sewi en jan anpa li kama lon tenpo sama.  
- likely misspelled word: temp  
- http://tatoeba.org/eng/sentences/show/9117066  

jan li jo e mani pi mute ili la jan li jo e jan lili pi mute pona.  
- likely misspelled word: ili  
- http://tatoeba.org/eng/sentences/show/9120460  



***
 

### Likely Error report for user Rodrigopredja


tenpo pimeja pini la mi sitelen lape e sina uta tawa uta mi  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/4975267  



***
 

### Likely Error report for user Zyurs


jan Mike ken ala pali e ijo sama ni.  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/6403400  

tenpo ali la jan ken pali e pali pi jan ante.  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/6485448  

ilo lukin ona li pakala  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6589582  

jan pi kalama musi li lon ala  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6589637  



***
 

### Likely Error report for user jan_Ne


kiwen musi ni li pona tawa mi  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6864631  

o tawa  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6864665  

mi tawa  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6864671  

mi toki kepeken toki Epanja  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6864683  

mi toki kepeken toki Inli  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6864685  

ma Mewika li lon poka ma Kanata  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/6864692  

ike ale ni li tan mi  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/7758436  

jan Tan li pana e suwi pimeja tawa jan Linta  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/7778509  

pipi li lon tenpo lili  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/7785521  

o kama lon poka mi  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/7893457  

pona tawa sina  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/8129773  



***
 

### Likely Error report for user Goatstein


len noka ni lli len kiwen.  
- likely misspelled word: lli  
- http://tatoeba.org/eng/sentences/show/7775311  

tenpo pini la, jan Ton ken ala lukin e jan.  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/7792134  

tenpo pini la, mi kama sona e ni: ma li tawa wawa lon ma tomo Awaji.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7792671  

duplicate please flag  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/7792698  

duplicate please flag  
- likely misspelled word: duplicate  
- http://tatoeba.org/eng/sentences/show/7792698  

duplicate please flag  
- likely misspelled word: please  
- http://tatoeba.org/eng/sentences/show/7792698  

duplicate please flag  
- likely misspelled word: flag  
- http://tatoeba.org/eng/sentences/show/7792698  

ona li weka e kulupu mama ona li tawa lon ma Tajiti.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7797653  

tenpo pimjea wan la, mi mute li ken ala ken awen lon tomo sina?  
- likely misspelled word: pimjea  
- http://tatoeba.org/eng/sentences/show/7797765  

nimi "anticonstitutionnellement" li jo e nimi lili luka luka luka luka luka li nimi suli mute pi toki Kanse.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7798710  

mama pi tenpo pini li sewi tawa kulupi ni.  
- likely misspelled word: kulupi  
- http://tatoeba.org/eng/sentences/show/7799863  

tenpo pini la, mi moku mute. mi pana e mani lili  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/7805078  

tenpo suno al la, seli li suli mute.  
- likely misspelled word: al  
- http://tatoeba.org/eng/sentences/show/7806808  

teno suno pini ni la, lon tenpo ali la, ona li moku e telo nasa.  
- likely misspelled word: teno  
- http://tatoeba.org/eng/sentences/show/7808296  

tenpo seme la sina li weka e lon ni?  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/7810236  

tenpo suno kama ni la, seli e mute seme?  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/7815138  

jan Ton li weka e ino tenpo ona.  
- likely misspelled word: ino  
- http://tatoeba.org/eng/sentences/show/7818266  

jan Peti o, ni li mije lili sina anu seme?  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7827545  

jan Ton il wile moli e sina tan seme?  
- likely misspelled word: il  
- http://tatoeba.org/eng/sentences/show/7833611  

poki li jo e kili liji loje mute.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7834553  

poki li jo e kili liji loje mute.  
- likely misspelled word: liji  
- http://tatoeba.org/eng/sentences/show/7834553  

jan Peti li telo e kasi kule.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7836199  

awen lon kulupu linja li ike mute awa mi.  
- likely misspelled word: awa  
- http://tatoeba.org/eng/sentences/show/7840617  

sike suno ni la, mi mute li jo e ko lete mute. sike suno pini la, mi mute li joe ko lete lili.  
- likely misspelled word: joe  
- http://tatoeba.org/eng/sentences/show/7842362  

tenpo pini lila la, mi toki e ona tawa jan ton.  
- likely misspelled word: lila  
- http://tatoeba.org/eng/sentences/show/7866735  

tenpo pini lila la, mi toki e ona tawa jan ton.  
- likely misspelled word: ton  
- http://tatoeba.org/eng/sentences/show/7866735  

jan Pati li kalama musi li tawa musi.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7868365  

jan Kanti li toki lon.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7868502  

ma Awaji li ma pona kin.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7868504  

tenpo ijo la, sina li seli ala seli e ilo nasa pi kiwen wawa?  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/7869710  

mi wile tawa ma Awaiji.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7956237  

jan Sami li lon ma tomo Pakati.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/7977227  

tenpo kin la, o lukin tawa e moku sama lon telo suli ali. kala ali li moku e kali ali, li utala ali tan tenpo pi ali open.  
- likely misspelled word: kali  
- http://tatoeba.org/eng/sentences/show/8001227  

sine wile e ijo seme?  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/8069210  

sine wile e ijo seme?  
- likely misspelled word: sine  
- http://tatoeba.org/eng/sentences/show/8069210  

jan mute li sona e sitelen jan Tatinka pi jan Lotin.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/8084875  

nimi mi li nimi Osimantija. mi jan lawa pi jan lawa. jan wawa o, lukin e pali mi, o pilin anpa a!  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/8086711  

mi pilin e ni: musi Kantiku en musi sama li ike tawa tenpo.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/8104755  

toki Alape non e ike mute.  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/8115071  

toki Alape non e ike mute.  
- likely misspelled word: non  
- http://tatoeba.org/eng/sentences/show/8115071  



***
 

### Likely Error report for user Ilia2


jan pi ma Losi li nimi "ъ" sama "твёрдый знак". ona li nimi "ь" sama "мягкий знак".  
- likely misspelled word: твёрдый  
- http://tatoeba.org/eng/sentences/show/7887101  

jan pi ma Losi li nimi "ъ" sama "твёрдый знак". ona li nimi "ь" sama "мягкий знак".  
- likely misspelled word: знак  
- http://tatoeba.org/eng/sentences/show/7887101  

jan pi ma Losi li nimi "ъ" sama "твёрдый знак". ona li nimi "ь" sama "мягкий знак".  
- likely misspelled word: мягкий  
- http://tatoeba.org/eng/sentences/show/7887101  

jan pi ma Losi li nimi "ъ" sama "твёрдый знак". ona li nimi "ь" sama "мягкий знак".  
- likely misspelled word: знак  
- http://tatoeba.org/eng/sentences/show/7887101  



***
 

### Likely Error report for user epiceric9


ona li weka e kulupu mama ona li tawa lon ma Tawiti.  
- forbidden syllables wu/wo/ji/ti  
- http://tatoeba.org/eng/sentences/show/8116919  



***
 

### Likely Error report for user shou


pona  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/8719275  



***
 

### Likely Error report for user Aleks


Tom ĉiutage lavas siajn dentojn.  
- likely misspelled word: ĉiutage  
- http://tatoeba.org/eng/sentences/show/9083237  

Tom ĉiutage lavas siajn dentojn.  
- likely misspelled word: lavas  
- http://tatoeba.org/eng/sentences/show/9083237  

Tom ĉiutage lavas siajn dentojn.  
- likely misspelled word: siajn  
- http://tatoeba.org/eng/sentences/show/9083237  

Tom ĉiutage lavas siajn dentojn.  
- likely misspelled word: dentojn  
- http://tatoeba.org/eng/sentences/show/9083237  

tenpo pini la, mi lukin e jan lawa meli  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/9085360  

sina li pilin pona anu seme?  
- mi\sina li  
- http://tatoeba.org/eng/sentences/show/9085472  



***
 

### Likely Error report for user Yl


mi tan ma Sonko  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/9131306  

jan Ale li tonsi  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/9131889  

lipu ni li jo la, jan tu li ken kama insa  
- Sentence doesn't end with punctuation  
- http://tatoeba.org/eng/sentences/show/9132036  



***
 

### Likely Error report for user eltacodemex


sina toki e toki katala, anu seme?  
- likely misspelled word: katala  
- http://tatoeba.org/eng/sentences/show/9178220  



***
 

### Likely Error report for user Sobsz


nimi kulupu ni ken wile ala ni: jan pali e ni tawa toki ante.  
- 'e' without 'li'/'mi'/'sina'/'o'  
- http://tatoeba.org/eng/sentences/show/9317512  

sina ken ala powe e mi.  
- likely misspelled word: powe  
- http://tatoeba.org/eng/sentences/show/9331471  



***
 

### Likely Error report for user inkepa


Look! The cat’s in the kitchen!  
- likely misspelled word: cat  
- http://tatoeba.org/eng/sentences/show/9343876  

Look! The cat’s in the kitchen!  
- likely misspelled word: s  
- http://tatoeba.org/eng/sentences/show/9343876  

Look! The cat’s in the kitchen!  
- likely misspelled word: in  
- http://tatoeba.org/eng/sentences/show/9343876  

Look! The cat’s in the kitchen!  
- likely misspelled word: the  
- http://tatoeba.org/eng/sentences/show/9343876  

Look! The cat’s in the kitchen!  
- likely misspelled word: kitchen  
- http://tatoeba.org/eng/sentences/show/9343876  

ona li jan pi esun kali.  
- likely misspelled word: kali  
- http://tatoeba.org/eng/sentences/show/9344097  


In [8]:

with open('/content/output.MD','w') as writefile: 
 writefile.write(output)
print("file written")


file written


In [5]:
with open('drive/MyDrive/tatoeba/test.txt', 'w') as writeFile:
  writeFile.write("test")